In [119]:
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.


In [120]:
import pandas_datareader.data as web
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)

In [5]:
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 

In [6]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [7]:
pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

In [9]:
from pypfopt.efficient_frontier import EfficientFrontier

In [10]:
import numpy as np

# DIA Stock Discovery

Just get rid of each stock and evaluate their changed performance

In [54]:
# call the function and generate a table with stock adjusted close prices
# Function for computing portfolio return
def portfolio_returns(df,weights,time):
    return (np.sum(df.mean() * weights)) * time
# Function for computing standard deviation of portfolio returns
def portfolio_sd(df,weights,time):
    return np.sqrt(np.transpose(weights) @ (df.cov() * time) @ weights)
def sharpe_fun(df,weights,time,rf):
    return ((portfolio_returns(df,weights,time)-rf) / portfolio_sd(df,weights,time))

def getrid(stocks,time1):
    results_return=pd.DataFrame(columns=stocks)
    #results_volatility=pd.DataFrame(columns=stocks)
    #results_sharpe=pd.DataFrame(columns=stocks)
    for i in range(len(stocks)):
        stock=stocks[0:i]+stocks[i+1:]
        portfolio=combine_stocks(stock)
        # calculate the expected return using historical data
        mu = mean_historical_return(portfolio, returns_data=False)
        # calculate the covariance matrix
        S = CovarianceShrinkage(portfolio).ledoit_wolf()
        from pypfopt.efficient_frontier import EfficientFrontier
        # generate the efficient frontier
        ef = EfficientFrontier(mu,S)
        # maximize sharpe ratio to help us get the portfolio weights
        try:
            weights = ef.max_sharpe()
            break
        except:
            weights=0
            
        if weights!=0:
            cleaned_weights=list(cleaned_weights.values())
            returns=portfolio_returns(df,cleaned_weights,time1)
            sd=portfolio_sd(df,cleaned_weights,time1)
            sharpe=sharpe_fun(df,cleaned_weights,time1,0.0298)
            result=[returns,sd,sharpe]
            results_return[results_return.columns[1]]=np.array(result) 
        else:
            results_return[results_return.columns[i]]=[0,0,0]
    return results_return
        
        

In [58]:
# call the function and generate a table with stock adjusted close prices
# Function for computing portfolio return
def portfolio_returns(df,weights,time):
    return (np.sum(df.mean() * weights)) * time
# Function for computing standard deviation of portfolio returns
def portfolio_sd(df,weights,time):
    return np.sqrt(np.transpose(weights) @ (df.cov() * time) @ weights)
def sharpe_fun(df,weights,time,rf):
    return ((portfolio_returns(df,weights,time)-rf) / portfolio_sd(df,weights,time))

def getrid(stocks,time1):
    results_return=pd.DataFrame(columns=stocks)
    #results_volatility=pd.DataFrame(columns=stocks)
    #results_sharpe=pd.DataFrame(columns=stocks)
    for i in range(len(stocks)):
        stock=stocks[0:i]+stocks[i+1:]
        portfolio=combine_stocks(stock)
        # calculate the expected return using historical data
        mu = mean_historical_return(portfolio, returns_data=False)
        # calculate the covariance matrix
        S = CovarianceShrinkage(portfolio).ledoit_wolf()
        from pypfopt.efficient_frontier import EfficientFrontier
        # generate the efficient frontier
        ef = EfficientFrontier(mu,S)
        # maximize sharpe ratio to help us get the portfolio weights
        weights = ef.max_sharpe()
            
        cleaned_weights=list(cleaned_weights.values())
        returns=portfolio_returns(df,cleaned_weights,time1)
        sd=portfolio_sd(df,cleaned_weights,time1)
        sharpe=sharpe_fun(df,cleaned_weights,time1,0.0298)
        result=[returns,sd,sharpe]
        results_return[results_return.columns[1]]=np.array(result) 
    return results_return
        
        

In [55]:
stocks=["UNH", "GS", "HD","MSFT"]
getrid(stocks,20)

                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  305.868286
2022-04-20  313.162262
2022-04-21  308.412231
2022-04-22  298.226532
2022-04-25  303.026245
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564056
2022-04-22  272.814178
2022-04-25  279.474487
                   UNH
Date                  
2022-04-19  535.789307
2022-04-20  544.069763
2022-04-21  535.530212
2022-04-22  519.088867
2022-04-25  522.407043
                    HD
Date                  
2022-04-19  305.868286
2022-04-20  313.162262
2022-04-21  308.412231
2022-04-22  298.226532
2022-04-25  303.026245
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564087
2022-04-22  272.814178
2022-04-25  279.474487
                   UNH
Date       

,UNH,GS,HD,MSFT
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0


In [ ]:
stocks = ["GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
portfolio=combine_stocks(stocks)
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
from pypfopt.efficient_frontier import EfficientFrontier
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()

In [59]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
table=getrid(stocks,20)

                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  305.868286
2022-04-20  313.162262
2022-04-21  308.412231
2022-04-22  298.226532
2022-04-25  303.026245
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564087
2022-04-22  272.814178
2022-04-25  279.474487
                   MCD
Date                  
2022-04-19  253.992325
2022-04-20  254.439850
2022-04-21  253.843155
2022-04-22  248.791153
2022-04-25  251.486221
                  AMGN
Date                  
2022-04-19  249.424332
2022-04-20  251.403046
2022-04-21  251.481796
2022-04-22  246.451370
2022-04-25  248.243027
                   CAT
Date                  
2022-04-19  229.453812
2022-04-20  232.339767
2022-04-21  229.859039
2022-04-22  214.803894
2022-04-25  215.201126
                     V
Date       

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [60]:
table

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC


In [61]:
results_return=pd.DataFrame(columns=stocks)

In [107]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:23]+stocks[24:]
portfolio=combine_stocks(stock)  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return[results_return.columns[23]]=np.array(result) 

                   UNH
Date                  
2022-04-19  535.789307
2022-04-20  544.069763
2022-04-21  535.530212
2022-04-22  519.088867
2022-04-25  522.407043
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  305.868286
2022-04-20  313.162262
2022-04-21  308.412231
2022-04-22  298.226532
2022-04-25  303.026245
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564056
2022-04-22  272.814178
2022-04-25  279.474487
                   MCD
Date                  
2022-04-19  253.992325
2022-04-20  254.439850
2022-04-21  253.843155
2022-04-22  248.791153
2022-04-25  251.486221
                  AMGN
Date                  
2022-04-19  249.424332
2022-04-20  251.403046
2022-04-21  251.481796
2022-04-22  246.451370
2022-04-25  248.243027
                   CAT
Date       

In [108]:
#results_return[results_return.columns[0]]=np.array(result)    
results_return

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,0.066596,NaN,0.025709,NaN,NaN,NaN,NaN,NaN,NaN
1,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,0.066381,NaN,0.076777,NaN,NaN,NaN,NaN,NaN,NaN
2,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,0.554318,NaN,-0.053289,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
results_return.to_csv("DIAdeletemean.csv", index=True)

Replace one of the stock with 10 year treasure

In [109]:
results_return2=pd.DataFrame(columns=stocks)

In [156]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=['^TNX']+stocks[1:]
stock=stocks[0:29]+['^TNX']+stocks[30:]
portfolio=combine_stocks(stock)  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return2[results_return2.columns[29]]=np.array(result) 

                   UNH
Date                  
2022-04-19  537.700012
2022-04-20  546.010010
2022-04-21  537.440002
2022-04-22  520.940002
2022-04-25  524.270020
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  307.799988
2022-04-20  315.140015
2022-04-21  310.359985
2022-04-22  300.109985
2022-04-25  304.940002
                  MSFT
Date                  
2022-04-19  285.299988
2022-04-20  286.359985
2022-04-21  280.809998
2022-04-22  274.029999
2022-04-25  280.720001
                   MCD
Date                  
2022-04-19  255.399994
2022-04-20  255.850006
2022-04-21  255.250000
2022-04-22  250.169998
2022-04-25  252.880005
                  AMGN
Date                  
2022-04-19  253.369995
2022-04-20  255.380005
2022-04-21  255.460007
2022-04-22  250.350006
2022-04-25  252.169998
                   CAT
Date       

In [157]:
results_return2

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.063701,0.063656,0.063701,0.063648,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063672,0.063701,0.063612,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.024591,0.063701,0.063701,0.063701,0.063701,0.063701,0.063657
1,0.064929,0.064886,0.064929,0.064877,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064901,0.064929,0.064842,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.075022,0.064929,0.064929,0.064929,0.064929,0.064929,0.064887
2,0.522119,0.521785,0.522119,0.521724,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.521905,0.522119,0.521456,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,-0.069435,0.522119,0.522119,0.522119,0.522119,0.522119,0.521794


In [158]:
results_return2.to_csv("DIAswapmeanmonth.csv", index=True)

In [123]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:2]+stocks[3:]
portfolio=combine_stocks(stock)  
t=len(portfolio.iloc[0:])
num=[]
for i in range(t):
    num.append(1)
    
portfolio['extra']=num
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
#results_return2[results_return2.columns[2]]=np.array(result) 

                   UNH
Date                  
2022-04-19  535.789307
2022-04-20  544.069763
2022-04-21  535.530212
2022-04-22  519.088867
2022-04-25  522.407043
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564087
2022-04-22  272.814178
2022-04-25  279.474487
                   MCD
Date                  
2022-04-19  253.992325
2022-04-20  254.439850
2022-04-21  253.843155
2022-04-22  248.791153
2022-04-25  251.486221
                  AMGN
Date                  
2022-04-19  249.424332
2022-04-20  251.403030
2022-04-21  251.481796
2022-04-22  246.451370
2022-04-25  248.243027
                   CAT
Date                  
2022-04-19  229.453812
2022-04-20  232.339783
2022-04-21  229.859039
2022-04-22  214.803894
2022-04-25  215.201126
                     V
Date       

Test risk parity portfolio while deleting each stock

In [165]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import inv,pinv
from scipy.optimize import minimize


import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
from scipy.optimize import minimize
TOLERANCE = 1e-10


def _allocation_risk(weights, covariances):

    # We calculate the portfolio risk (sigma^2)
    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]  

    # It returns the risk of the weights distribution
    return portfolio_risk


def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    # We calculate the risk of the portfolio
    portfolio_risk = _allocation_risk(weights, covariances)

    # We calculate the contribution of each asset to the risk of total portfolio risk
    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T)/portfolio_risk

    # It returns the contribution of each asset to the risk of the weights
    # distribution
    return assets_risk_contribution


def _risk_budget_objective_error(weights, args):

    # The covariance matrix occupies the first position in the variable
    covariances = args[0]

    # The desired contribution of each asset to the portfolio risk occupies the
    # second position
    assets_risk_budget = args[1]

    # We convert the weights to a matrix
    weights = np.matrix(weights)

    # We calculate the portfolio risk (total portfolio variance)
    portfolio_risk = _allocation_risk(weights, covariances)

    # We calculate the contribution of each asset to the risk of the portfolio 
    assets_risk_contribution = _assets_risk_contribution_to_allocation_risk(weights, covariances)

    # We calculate the desired contribution of each asset to the risk of the portfolio
    assets_risk_target = np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    # Error between the desired contribution and the calculated/actual contribution of
    # each asset
    error = sum(np.square(assets_risk_contribution - assets_risk_target.T))[0, 0]
    
    # It returns the calculated error
    return error


def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    # Restrictions to consider in the optimisation: only long positions whose
    # sum equals 100%
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})

    # Optimisation process in scipy
    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    # Recover the weights from the optimised object
    weights = optimize_result.x

    # It returns the optimised weights
    return weights


def get_weights(df,portfolio):

    # We download the prices from Yahoo Finance
    prices = portfolio

    # We calculate the covariance matrix
    #covariances = 252.0 * prices.asfreq('W-FRI').pct_change().iloc[1:, :].cov().values
    covariances=np.matrix(df.cov())

    # The desired contribution of each asset to the portfolio risk: we want all
    # asset to contribute equally
    assets_risk_budget = [1 / prices.shape[1]] * prices.shape[1]

    # Initial weights: equally weighted
    #init_weights = [1 / prices.shape[1]] * prices.shape[1]
    init_weights=[]
    sum=0
    for i in range(len(portfolio.columns)-1):
        init_weights.append(0.1)
        sum+=0.1
    init_weights.append(1-sum)
    # Optimisation process of weights
    weights =  _get_risk_parity_weights(covariances, assets_risk_budget, init_weights)

    # Convert the weights to a pandas Series
    weights = pd.Series(weights, index=prices.columns, name='weight')

    # It returns the optimised weights
    return weights


In [161]:
results_return3=pd.DataFrame(columns=stocks)

In [210]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:20]+stocks[21:]
portfolio=combine_stocks(stock)   
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio)
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return3[results_return3.columns[20]]=np.array(result) 

                   UNH
Date                  
2022-04-19  535.789307
2022-04-20  544.069763
2022-04-21  535.530212
2022-04-22  519.088867
2022-04-25  522.407043
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  305.868286
2022-04-20  313.162262
2022-04-21  308.412231
2022-04-22  298.226532
2022-04-25  303.026245
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564056
2022-04-22  272.814178
2022-04-25  279.474487
                   MCD
Date                  
2022-04-19  253.992325
2022-04-20  254.439850
2022-04-21  253.843155
2022-04-22  248.791153
2022-04-25  251.486221
                  AMGN
Date                  
2022-04-19  249.424332
2022-04-20  251.403030
2022-04-21  251.481796
2022-04-22  246.451370
2022-04-25  248.243027
                   CAT
Date       

In [211]:
results_return3

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,-0.082755,-0.083995,-0.085926,-0.084168,-0.085355,-0.088685,-0.084941,-0.085673,-0.088007,-0.087749,-0.083706,-0.088513,-0.087624,-0.083728,-0.087569,-0.084023,-0.087148,-0.078876,-0.084871,-0.072368,-0.082986,-0.082396,-0.081308,-0.093728,-0.085048,-0.090182,-0.085097,-0.082905,-0.084629,-0.084747
1,0.066396,0.066113,0.065971,0.065863,0.066741,0.070414,0.066336,0.066209,0.066833,0.069122,0.065612,0.067824,0.066510,0.066249,0.066319,0.065849,0.066562,0.066313,0.066875,0.068531,0.066342,0.065739,0.066769,0.068335,0.066427,0.067262,0.067587,0.065850,0.065989,0.065774
2,-1.695206,-1.721212,-1.754182,-1.730382,-1.725398,-1.682693,-1.729690,-1.744056,-1.762713,-1.700615,-1.729951,-1.744418,-1.765505,-1.713661,-1.769779,-1.728533,-1.756984,-1.638844,-1.714717,-1.490831,-1.700056,-1.706687,-1.664065,-1.807684,-1.728953,-1.783800,-1.699982,-1.711543,-1.734069,-1.741524


In [212]:
results_return3.to_csv("DIAdeleteriskmonth.csv", index=True)

evaluate DIA risk-parity effect using stock swapping with 10 year treasury

In [219]:
results_return4=pd.DataFrame(columns=stocks)

In [253]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=['^TNX']+stocks[1:]
stock=stocks[0:4]+['^TNX']+stocks[5:]
portfolio=combine_stocks(stock)  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio)
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return4[results_return4.columns[4]]=np.array(result) 

                   UNH
Date                  
2022-04-19  537.700012
2022-04-20  546.010010
2022-04-21  537.440002
2022-04-22  520.940002
2022-04-25  524.270020
                    GS
Date                  
2022-04-19  335.950012
2022-04-20  341.059998
2022-04-21  334.299988
2022-04-22  319.769989
2022-04-25  321.420013
                    HD
Date                  
2022-04-19  307.799988
2022-04-20  315.140015
2022-04-21  310.359985
2022-04-22  300.109985
2022-04-25  304.940002
                  MSFT
Date                  
2022-04-19  285.299988
2022-04-20  286.359985
2022-04-21  280.809998
2022-04-22  274.029999
2022-04-25  280.720001
             ^TNX
Date             
2022-04-19  2.913
2022-04-20  2.840
2022-04-21  2.917
2022-04-22  2.906
2022-04-25  2.826
                  AMGN
Date                  
2022-04-19  253.369995
2022-04-20  255.380005
2022-04-21  255.460007
2022-04-22  250.350006
2022-04-25  252.169998
                   CAT
Date                  
2022-04-19  232.160004


In [254]:
results_return4

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,-0.075483,-0.076201,-0.076642,-0.073403,-0.077521,-0.079495,-0.076209,-0.077643,-0.079147,-0.080199,-0.076322,-0.079920,-0.079072,-0.075722,-0.080335,-0.076150,-0.079955,-0.071621,-0.077014,-0.063881,-0.077455,-0.073040,-0.071392,-0.086084,-0.078590,-0.081942,-0.074714,-0.075792,-0.077642,-0.075831
1,0.060728,0.060388,0.060165,0.058651,0.061070,0.064275,0.060570,0.060484,0.061142,0.063297,0.060079,0.062019,0.060778,0.060517,0.060809,0.060161,0.060979,0.060582,0.060932,0.061193,0.060856,0.058526,0.059250,0.062533,0.060860,0.061352,0.060168,0.060121,0.060447,0.060069
2,-1.733671,-1.755323,-1.769156,-1.759629,-1.757340,-1.700442,-1.750200,-1.776393,-1.781879,-1.737815,-1.766388,-1.769117,-1.791308,-1.743673,-1.811162,-1.761103,-1.799893,-1.674117,-1.752989,-1.530907,-1.762421,-1.757161,-1.707873,-1.853151,-1.780959,-1.821326,-1.737037,-1.756334,-1.777456,-1.758479


In [255]:
results_return4.to_csv("DIAswapriskmonth.csv", index=True)

quarterly

In [256]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

mean-variance by getting rid of one stock

In [257]:
results_return5=pd.DataFrame(columns=stocks)

In [291]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:29]+stocks[30:]
portfolio=combine_stocks(stock)  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
results_return5[results_return5.columns[29]]=np.array(result) 

                   UNH
Date                  
2021-12-20  483.730011
2021-12-21  487.209991
2021-12-22  494.130005
2021-12-23  495.380005
2021-12-27  499.500000
                    GS
Date                  
2021-12-20  369.432312
2021-12-21  378.091309
2021-12-22  380.089508
2021-12-23  382.783630
2021-12-27  385.766052
                    HD
Date                  
2021-12-20  389.399994
2021-12-21  390.470001
2021-12-22  395.640015
2021-12-23  397.070007
2021-12-27  404.089996
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  219.990005
2021-12-21  220.919998
2021-12-22  221.529999
2021-12-23  223.789993
2021-12-27  225.169998
                   CAT
Date       

In [292]:
results_return5

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.239355,0.239856,0.239240,0.239749,0.239464,0.239335,0.239149,0.241089,0.239948,0.240036,0.239466,0.240385,0.157273,0.233591,0.239564,0.239740,0.239445,0.240387,0.239399,0.252468,0.239767,0.239944,0.239638,0.239369,0.247075,0.239328,0.238914,0.238865,0.238819,0.240182
1,0.071504,0.071670,0.071466,0.071634,0.071540,0.071497,0.072214,0.073650,0.072478,0.072204,0.071622,0.074208,0.088072,0.071369,0.072441,0.071713,0.072402,0.072320,0.072387,0.078297,0.072509,0.072567,0.071598,0.071509,0.075437,0.071495,0.071731,0.071797,0.071781,0.072163
2,2.930688,2.930883,2.930639,2.930847,2.930729,2.930674,2.898984,2.868843,2.899469,2.911708,2.927401,2.837758,1.447372,2.855475,2.895631,2.927507,2.895559,2.911879,2.895529,2.843902,2.895747,2.895855,2.930795,2.930690,2.880229,2.930673,2.915247,2.911895,2.911884,2.915395


In [293]:
results_return5.to_csv("DIAdeletemeanyear.csv", index=True)

In [359]:
portfolio=combine_stocks(stocks)

                   UNH
Date                  
2021-12-20  480.588898
2021-12-21  484.046295
2021-12-22  490.921387
2021-12-23  492.163239
2021-12-27  496.256500
                    GS
Date                  
2021-12-20  367.183655
2021-12-21  375.789948
2021-12-22  377.775970
2021-12-23  380.453705
2021-12-27  383.417969
                    HD
Date                  
2021-12-20  384.635010
2021-12-21  385.691895
2021-12-22  390.798645
2021-12-23  392.211151
2021-12-27  399.145233
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  214.723099
2021-12-21  215.630798
2021-12-22  216.226212
2021-12-23  218.432098
2021-12-27  219.779068
                   CAT
Date       

In [294]:
results_return6=pd.DataFrame(columns=stocks)

In [360]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
portfolio=combine_stocks(stocks)

                   UNH
Date                  
2021-12-20  480.588898
2021-12-21  484.046295
2021-12-22  490.921356
2021-12-23  492.163239
2021-12-27  496.256500
                    GS
Date                  
2021-12-20  367.183655
2021-12-21  375.789948
2021-12-22  377.775970
2021-12-23  380.453705
2021-12-27  383.417969
                    HD
Date                  
2021-12-20  384.635010
2021-12-21  385.691895
2021-12-22  390.798645
2021-12-23  392.211151
2021-12-27  399.145233
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  214.723068
2021-12-21  215.630798
2021-12-22  216.226212
2021-12-23  218.432098
2021-12-27  219.779053
                   CAT
Date       

In [390]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:29]+stocks[30:]
portfolio1=portfolio[stock]  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
results_return6[results_return6.columns[29]]=np.array(result) 

In [401]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 

In [332]:
result

[0.08498629527217097, 0.04150730728347753, 1.3295561404470706]

In [402]:
results_return6

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.239355,0.239856,0.239240,0.239749,0.239464,0.239328,0.238838,0.240763,0.239618,0.239370,0.239619,0.240621,0.157270,0.234167,0.239456,0.239881,0.239333,0.239711,0.239301,0.253367,0.239673,0.239808,0.239638,0.239369,0.247075,0.239328,0.239388,0.239543,0.239468,0.239301
1,0.071504,0.071670,0.071466,0.071634,0.071540,0.071495,0.071333,0.072633,0.071591,0.071509,0.071591,0.074194,0.088069,0.070525,0.071537,0.071678,0.071497,0.071622,0.071486,0.077221,0.071609,0.071654,0.071598,0.071509,0.075437,0.071495,0.071515,0.071566,0.071542,0.071486
2,2.930687,2.930884,2.930638,2.930846,2.930730,2.930673,2.930442,2.904531,2.930789,2.930696,2.930785,2.841482,1.447386,2.897807,2.930729,2.930884,2.930664,2.930816,2.930661,2.895158,2.930816,2.930861,2.930795,2.930690,2.880228,2.930673,2.930700,2.930757,2.930722,2.930661


In [403]:
results_return6.to_csv("DIAswapmeanquarter.csv", index=True)

test risk parity for quarter and by deletion of stock

In [336]:
results_return7=pd.DataFrame(columns=stocks)

In [393]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
portfolio2=combine_stocks(stocks)

                   UNH
Date                  
2021-12-20  480.588898
2021-12-21  484.046295
2021-12-22  490.921387
2021-12-23  492.163239
2021-12-27  496.256500
                    GS
Date                  
2021-12-20  367.183655
2021-12-21  375.789948
2021-12-22  377.775970
2021-12-23  380.453705
2021-12-27  383.417969
                    HD
Date                  
2021-12-20  384.634979
2021-12-21  385.691895
2021-12-22  390.798645
2021-12-23  392.211151
2021-12-27  399.145233
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037140
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  214.723083
2021-12-21  215.630814
2021-12-22  216.226212
2021-12-23  218.432098
2021-12-27  219.779053
                   CAT
Date       

In [394]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 

In [395]:
results_return7

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.023566,0.026917,0.029751,0.026384,0.029163,0.022273,0.019495,0.024064,0.025863,0.023470,0.026590,0.018869,0.011019,0.022439,0.024592,0.025781,0.031319,0.025071,0.027847,0.023398,0.027728,0.028679,0.026344,0.023447,0.023675,0.019206,0.026921,0.027906,0.026581,0.022720
1,0.068144,0.067405,0.068034,0.067638,0.068878,0.068613,0.069179,0.067551,0.067427,0.069347,0.067285,0.068458,0.071560,0.067139,0.068754,0.067380,0.067952,0.067142,0.068920,0.070450,0.067438,0.067399,0.067888,0.069827,0.069313,0.067726,0.070027,0.067785,0.068076,0.070619
2,-0.091484,-0.042771,-0.000724,-0.050509,-0.009255,-0.109709,-0.148958,-0.084914,-0.058389,-0.091286,-0.047703,-0.159679,-0.262448,-0.109643,-0.075747,-0.059648,0.022348,-0.070435,-0.028342,-0.090872,-0.030721,-0.016636,-0.050902,-0.090977,-0.088371,-0.156427,-0.041117,-0.027943,-0.047279,-0.100252


In [338]:
results_return7

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.007855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.039343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.557779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [396]:
results_return7.to_csv("DIAdeleteriskquarter.csv", index=True)

In [400]:
results_return8=pd.DataFrame(columns=stocks)

In [404]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 

In [405]:
results_return8

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.023566,0.026917,0.029751,0.026384,0.029163,0.022273,0.019495,0.024064,0.025863,0.023470,0.026590,0.018869,0.011019,0.022439,0.024592,0.025781,0.031319,0.025071,0.027847,0.023398,0.027728,0.028679,0.026344,0.023447,0.023675,0.019206,0.026921,0.027906,0.026581,0.022720
1,0.068144,0.067405,0.068034,0.067638,0.068878,0.068613,0.069179,0.067551,0.067427,0.069347,0.067285,0.068458,0.071560,0.067139,0.068754,0.067380,0.067952,0.067142,0.068920,0.070450,0.067438,0.067399,0.067888,0.069827,0.069313,0.067726,0.070027,0.067785,0.068076,0.070619
2,-0.091485,-0.042771,-0.000724,-0.050510,-0.009255,-0.109709,-0.148958,-0.084914,-0.058389,-0.091286,-0.047704,-0.159679,-0.262448,-0.109643,-0.075747,-0.059648,0.022348,-0.070435,-0.028342,-0.090872,-0.030722,-0.016636,-0.050902,-0.090977,-0.088371,-0.156427,-0.041117,-0.027943,-0.047279,-0.100252


In [406]:
results_return8.to_csv("DIAswapriskquarter.csv", index=True)

for yearly data!

In [482]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [483]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
portfolio=combine_stocks(stocks)

                   UNH
Date                  
2020-06-19  283.147247
2020-06-22  284.537537
2020-06-23  289.330536
2020-06-24  281.144501
2020-06-25  287.988861
                    GS
Date                  
2020-06-19  193.557953
2020-06-22  195.276291
2020-06-23  196.581848
2020-06-24  190.092499
2020-06-25  198.808975
                    HD
Date                  
2020-06-19  236.106979
2020-06-22  238.509735
2020-06-23  239.610550
2020-06-24  235.609222
2020-06-25  234.891281
                  MSFT
Date                  
2020-06-19  191.307037
2020-06-22  196.620331
2020-06-23  197.933914
2020-06-24  193.944107
2020-06-25  196.394836
                   MCD
Date                  
2020-06-19  178.266800
2020-06-22  179.126831
2020-06-23  178.324127
2020-06-24  176.097733
2020-06-25  174.635727
                  AMGN
Date                  
2020-06-19  222.975250
2020-06-22  218.762360
2020-06-23  220.219589
2020-06-24  217.193008
2020-06-25  219.621750
                   CAT
Date       

In [484]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
portfolio2=combine_stocks(stocks)

                   UNH
Date                  
2020-06-19  283.147217
2020-06-22  284.537506
2020-06-23  289.330536
2020-06-24  281.144501
2020-06-25  287.988831
                    GS
Date                  
2020-06-19  193.557953
2020-06-22  195.276306
2020-06-23  196.581833
2020-06-24  190.092484
2020-06-25  198.808990
                    HD
Date                  
2020-06-19  236.106995
2020-06-22  238.509720
2020-06-23  239.610550
2020-06-24  235.609207
2020-06-25  234.891296
                  MSFT
Date                  
2020-06-19  191.307037
2020-06-22  196.620346
2020-06-23  197.933929
2020-06-24  193.944092
2020-06-25  196.394867
                   MCD
Date                  
2020-06-19  178.266815
2020-06-22  179.126801
2020-06-23  178.324112
2020-06-24  176.097717
2020-06-25  174.635757
                  AMGN
Date                  
2020-06-19  222.975266
2020-06-22  218.762360
2020-06-23  220.219604
2020-06-24  217.193024
2020-06-25  219.621750
                   CAT
Date       

In [410]:
results_return9=pd.DataFrame(columns=stocks)

delete for mean variance

In [413]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result) 

In [414]:
results_return9

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.497421,0.451425,0.487520,0.491188,0.511987,0.487522,0.466569,0.487530,0.487552,0.487533,0.487449,0.487536,0.487530,0.487456,0.487535,0.468017,0.487540,0.487528,0.487522,0.487524,0.487521,0.487515,0.487204,0.487534,0.487530,0.487550,0.487532,0.487527,0.487518,0.487514
1,0.188852,0.174229,0.182898,0.184643,0.194156,0.182899,0.183786,0.182900,0.182905,0.182900,0.182880,0.182901,0.182900,0.182882,0.182901,0.179667,0.182903,0.182900,0.182897,0.182899,0.182898,0.182896,0.183790,0.182902,0.182900,0.182905,0.182900,0.182899,0.182897,0.182897
2,2.476127,2.419939,2.502602,2.498813,2.483502,2.502602,2.376507,2.502625,2.502673,2.502635,2.502451,2.502647,2.502625,2.502466,2.502635,2.439047,2.502638,2.502619,2.502622,2.502611,2.502598,2.502598,2.488733,2.502623,2.502627,2.502670,2.502635,2.502623,2.502595,2.502578


In [415]:
results_return9.to_csv("DIAdeletemeanyear.csv", index=True)

swap for the mean variance

In [416]:
results_return10=pd.DataFrame(columns=stocks)

In [418]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 

In [419]:
results_return10.to_csv("DIAswapmeanyear.csv", index=True)

test for risk-parity

delete testing

In [485]:
results_return11=pd.DataFrame(columns=stocks)

In [486]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result) 

In [422]:
results_return11

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC,^TNX
0,0.064993,0.063470,0.065837,0.064801,0.065947,0.067683,0.063671,0.066127,0.064638,0.066808,0.064757,0.065714,0.067457,0.065704,0.065813,0.063480,0.065946,0.064749,0.067041,0.066867,0.064388,0.065063,0.064326,0.067645,0.066286,0.065752,0.068030,0.066210,0.065855,0.067357,NaN
1,0.069375,0.068620,0.069786,0.068913,0.070249,0.069592,0.068746,0.068650,0.068399,0.070045,0.069117,0.068804,0.070698,0.070572,0.068957,0.068674,0.069611,0.067878,0.070560,0.070643,0.068637,0.069511,0.068896,0.070387,0.069353,0.070565,0.070595,0.069182,0.068748,0.068983,NaN
2,0.507287,0.490676,0.516391,0.507905,0.514555,0.544366,0.492701,0.529158,0.509334,0.528343,0.505760,0.521976,0.532646,0.508756,0.522255,0.490430,0.519259,0.514881,0.527801,0.524717,0.503925,0.507304,0.501132,0.537673,0.526086,0.509488,0.541530,0.526289,0.524455,0.544432,NaN


In [487]:
results_return11.to_csv("DIAdeleteriskyear.csv", index=True)

swap testings

In [488]:
results_return12=pd.DataFrame(columns=stocks)

In [489]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 

In [490]:
results_return12

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.288533,0.281725,0.291770,0.287059,0.292366,0.299298,0.282761,0.292658,0.286559,0.295771,0.280242,0.291187,0.291712,0.285539,0.292017,0.281332,0.292160,0.287635,0.296804,0.296106,0.285619,0.288605,0.285206,0.299080,0.293556,0.285654,0.300539,0.293462,0.291866,0.298783
1,0.144399,0.142394,0.145311,0.143524,0.145943,0.144731,0.142642,0.142694,0.142094,0.145530,0.141224,0.142822,0.142019,0.141697,0.143481,0.143044,0.144563,0.140733,0.146646,0.146720,0.142272,0.144437,0.143008,0.146149,0.144104,0.141767,0.146520,0.143986,0.142828,0.142802
2,1.791789,1.769207,1.802821,1.792447,1.799104,1.862064,1.773397,1.842118,1.806962,1.827609,1.773362,1.830154,1.844201,1.804833,1.827541,1.758419,1.814854,1.832078,1.820743,1.815063,1.798094,1.791826,1.785961,1.842506,1.830315,1.804750,1.847796,1.831163,1.834840,1.883612


In [491]:
results_return12.to_csv("DIAswapriskyear.csv", index=True)

# ALTY

monthly

In [428]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [429]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
portfolio=combine_stocks(stocks)

                 PFFD
Date                 
2022-04-19  21.726681
2022-04-20  21.922770
2022-04-21  21.667852
2022-04-22  21.442352
2022-04-25  21.540394
                SRET
Date                
2022-04-19  8.722801
2022-04-20  8.800943
2022-04-21  8.781408
2022-04-22  8.664192
2022-04-25  8.654424
                 QYLD
Date                 
2022-04-19  20.033581
2022-04-20  19.917162
2022-04-21  19.674625
2022-04-22  19.305965
2022-04-25  19.509699
                  MMP
Date                 
2022-04-19  50.004974
2022-04-20  50.917122
2022-04-21  50.245014
2022-04-22  49.092827
2022-04-25  48.430317
                   ED
Date                 
2022-04-19  96.429688
2022-04-20  96.823250
2022-04-21  96.606789
2022-04-22  94.934189
2022-04-25  94.501297
                  IEP
Date                 
2022-04-19  50.051765
2022-04-20  49.903355
2022-04-21  49.476669
2022-04-22  49.022160
2022-04-25  49.402462
                  EPD
Date                 
2022-04-19  26.074188
2022-04-20  26.23

In [438]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
portfolio2=combine_stocks(stocks)

                 PFFD
Date                 
2022-04-19  21.726679
2022-04-20  21.922768
2022-04-21  21.667852
2022-04-22  21.442352
2022-04-25  21.540394
                SRET
Date                
2022-04-19  8.722801
2022-04-20  8.800945
2022-04-21  8.781408
2022-04-22  8.664192
2022-04-25  8.654424
                 QYLD
Date                 
2022-04-19  20.033579
2022-04-20  19.917162
2022-04-21  19.674625
2022-04-22  19.305965
2022-04-25  19.509699
                  MMP
Date                 
2022-04-19  50.004978
2022-04-20  50.917122
2022-04-21  50.245014
2022-04-22  49.092827
2022-04-25  48.430317
                   ED
Date                 
2022-04-19  96.429688
2022-04-20  96.823250
2022-04-21  96.606789
2022-04-22  94.934189
2022-04-25  94.501297
                  IEP
Date                 
2022-04-19  50.051765
2022-04-20  49.903355
2022-04-21  49.476669
2022-04-22  49.022160
2022-04-25  49.402462
                  EPD
Date                 
2022-04-19  26.074190
2022-04-20  26.23

delete for mean variance

In [431]:
results_return1=pd.DataFrame(columns=stocks)

In [442]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return1[results_return1.columns[i]]=np.array(result) 

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [467]:
#stock=stocks[1:]
stock=stocks[0:18]+stocks[19:]
portfolio1=portfolio2[stock]  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return1[results_return1.columns[18]]=np.array(result) 

IndexError: index 18 is out of bounds for axis 0 with size 18

In [468]:
results_return1

,PFFD,SRET,QYLD,MMP,ED,IEP,EPD,HEP,SPH,SO,ETR,PPL,DUK,D,SUN,MPLX,OGE,EIX
0,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888,NaN,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888,0.015888
1,0.098672,0.098672,0.098672,0.098672,0.098672,0.098672,0.098671,0.098672,NaN,0.098672,0.098672,0.098672,0.098672,0.098672,0.098672,0.098671,0.098672,0.098671
2,-0.140990,-0.140990,-0.140990,-0.140990,-0.140990,-0.140990,-0.140993,-0.140990,NaN,-0.140990,-0.140990,-0.140990,-0.140990,-0.140990,-0.140990,-0.140993,-0.140990,-0.140993


In [469]:
results_return1.to_csv("ALTYdeletemeanmonth.csv", index=True)

swap for mean variance

In [470]:
results_return2=pd.DataFrame(columns=stocks)

In [472]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    if i!=8:
        weights = ef.max_sharpe()
        cleaned_weights = ef.clean_weights()  
        cleaned_weights=list(cleaned_weights.values())
        df=pd.DataFrame(columns=portfolio1.columns)
        for j in df:
            df[j]=portfolio1[j].pct_change()
        df=df.dropna()

        returns=portfolio_returns(df,cleaned_weights,20)
        sd=portfolio_sd(df,cleaned_weights,20)
        sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
        result=[returns,sd,sharpe]
        results_return2[results_return2.columns[i]]=np.array(result) 

In [473]:
results_return2

,PFFD,SRET,QYLD,MMP,ED,IEP,EPD,HEP,SPH,SO,ETR,PPL,DUK,D,SUN,MPLX,OGE,EIX
0,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197,NaN,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197,0.015197
1,0.096406,0.096406,0.096406,0.096406,0.096406,0.096405,0.096406,0.096406,NaN,0.096406,0.096406,0.096406,0.096406,0.096406,0.096406,0.096406,0.096406,0.096406
2,-0.151469,-0.151469,-0.151469,-0.151469,-0.151469,-0.151472,-0.151469,-0.151469,NaN,-0.151469,-0.151469,-0.151469,-0.151469,-0.151469,-0.151469,-0.151469,-0.151469,-0.151469


In [474]:
results_return2.to_csv("ALTYswapmeanmonth.csv", index=True)

delete for risk

In [475]:
results_return3=pd.DataFrame(columns=stocks)

In [477]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,30)
    sd=portfolio_sd(df,cleaned_weights,30)
    sharpe=sharpe_fun(df,cleaned_weights,30,0.0298)
    result=[returns,sd,sharpe]
    results_return3[results_return3.columns[i]]=np.array(result) 

In [478]:
results_return3.to_csv("ALTYdeleteriskmonth.csv", index=True)

In [479]:
results_return4=pd.DataFrame(columns=stocks)

In [480]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,30)
    sd=portfolio_sd(df,cleaned_weights,30)
    sharpe=sharpe_fun(df,cleaned_weights,30,0.0298)
    result=[returns,sd,sharpe]
    results_return4[results_return4.columns[i]]=np.array(result) 

In [481]:
results_return4.to_csv("ALTYswapriskmonth.csv", index=True)

quarterly

In [492]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [493]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
portfolio=combine_stocks(stocks)

                 PFFD
Date                 
2021-12-20  24.391523
2021-12-21  24.535967
2021-12-22  24.661152
2021-12-23  24.670780
2021-12-27  24.680410
                SRET
Date                
2021-12-20  8.649742
2021-12-21  8.879128
2021-12-22  8.926918
2021-12-23  8.984262
2021-12-27  9.070282
                 QYLD
Date                 
2021-12-20  20.115582
2021-12-21  20.389017
2021-12-22  20.534853
2021-12-23  20.616880
2021-12-27  20.726254
                  MMP
Date                 
2021-12-20  41.407120
2021-12-21  41.360111
2021-12-22  41.454128
2021-12-23  41.773804
2021-12-27  42.497772
                   ED
Date                 
2021-12-20  81.898132
2021-12-21  81.245209
2021-12-22  81.712975
2021-12-23  81.635010
2021-12-27  81.596031
                  IEP
Date                 
2021-12-20  44.659901
2021-12-21  44.302624
2021-12-22  44.624176
2021-12-23  44.436600
2021-12-27  44.516991
                  EPD
Date                 
2021-12-20  19.614305
2021-12-21  19.84

In [494]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
portfolio2=combine_stocks(stocks)

                 PFFD
Date                 
2021-12-20  24.391527
2021-12-21  24.535969
2021-12-22  24.661150
2021-12-23  24.670780
2021-12-27  24.680407
                SRET
Date                
2021-12-20  8.649742
2021-12-21  8.879128
2021-12-22  8.926917
2021-12-23  8.984263
2021-12-27  9.070282
                 QYLD
Date                 
2021-12-20  20.115585
2021-12-21  20.389019
2021-12-22  20.534849
2021-12-23  20.616880
2021-12-27  20.726255
                  MMP
Date                 
2021-12-20  41.407120
2021-12-21  41.360111
2021-12-22  41.454128
2021-12-23  41.773804
2021-12-27  42.497772
                   ED
Date                 
2021-12-20  81.898132
2021-12-21  81.245209
2021-12-22  81.712967
2021-12-23  81.635010
2021-12-27  81.596039
                  IEP
Date                 
2021-12-20  44.659901
2021-12-21  44.302620
2021-12-22  44.624176
2021-12-23  44.436604
2021-12-27  44.516991
                  EPD
Date                 
2021-12-20  19.614304
2021-12-21  19.84

delete for mean variance

In [495]:
results_return5=pd.DataFrame(columns=stocks)

In [496]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return5[results_return5.columns[i]]=np.array(result)

In [497]:
results_return5.to_csv("ALTYdeletemeanquarter.csv", index=True)

swap for mean variance

In [498]:
results_return6=pd.DataFrame(columns=stocks)

In [499]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 

In [500]:
results_return6.to_csv("ALTYswapmeanquarter.csv", index=True)

delete for risk parity

In [502]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]

results_return7=pd.DataFrame(columns=stocks)

In [503]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 

In [504]:
results_return7.to_csv("ALTYdeleteriskquarter.csv", index=True)

swap for risk

In [505]:
results_return8=pd.DataFrame(columns=stocks)

In [506]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 

In [507]:
results_return8.to_csv("ALTYswapriskquarter.csv", index=True)

yearly

In [508]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [510]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
portfolio=combine_stocks(stocks)

                 PFFD
Date                 
2020-06-19  21.187500
2020-06-22  21.098402
2020-06-23  21.116222
2020-06-24  21.053852
2020-06-25  21.036036
                SRET
Date                
2020-06-19  6.779766
2020-06-22  6.779766
2020-06-23  6.839916
2020-06-24  6.564945
2020-06-25  6.711023
                 QYLD
Date                 
2020-06-19  16.065613
2020-06-22  16.149923
2020-06-23  16.227272
2020-06-24  16.057106
2020-06-25  16.165396
                  MMP
Date                 
2020-06-19  36.099304
2020-06-22  36.205292
2020-06-23  35.952564
2020-06-24  34.591099
2020-06-25  34.794907
                   ED
Date                 
2020-06-19  65.543571
2020-06-22  65.525230
2020-06-23  65.084846
2020-06-24  65.626152
2020-06-25  64.791260
                  IEP
Date                 
2020-06-19  34.947250
2020-06-22  35.047920
2020-06-23  34.875347
2020-06-24  34.336037
2020-06-25  34.213791
                  EPD
Date                 
2020-06-19  15.857388
2020-06-22  16.09

In [511]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
portfolio2=combine_stocks(stocks)

                 PFFD
Date                 
2020-06-19  21.187500
2020-06-22  21.098404
2020-06-23  21.116220
2020-06-24  21.053850
2020-06-25  21.036032
                SRET
Date                
2020-06-19  6.779766
2020-06-22  6.779766
2020-06-23  6.839917
2020-06-24  6.564944
2020-06-25  6.711023
                 QYLD
Date                 
2020-06-19  16.065617
2020-06-22  16.149923
2020-06-23  16.227268
2020-06-24  16.057108
2020-06-25  16.165394
                  MMP
Date                 
2020-06-19  36.099308
2020-06-22  36.205296
2020-06-23  35.952557
2020-06-24  34.591095
2020-06-25  34.794910
                   ED
Date                 
2020-06-19  65.543579
2020-06-22  65.525223
2020-06-23  65.084862
2020-06-24  65.626144
2020-06-25  64.791252
                  IEP
Date                 
2020-06-19  34.947250
2020-06-22  35.047924
2020-06-23  34.875343
2020-06-24  34.336040
2020-06-25  34.213795
                  EPD
Date                 
2020-06-19  15.857386
2020-06-22  16.09

delete for mean variance

In [512]:
results_return9=pd.DataFrame(columns=stocks)

In [513]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result)

In [514]:
results_return9.to_csv("ALTYdeletemeanyear.csv", index=True)

swap for mean variance

In [515]:
results_return10=pd.DataFrame(columns=stocks)

In [516]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 

In [517]:
results_return10.to_csv("ALTYswapmeanyear.csv", index=True)

delete for risk

In [518]:
stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]

results_return11=pd.DataFrame(columns=stocks)

In [519]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result)

In [520]:
results_return11.to_csv("ALTYdeleteriskyear.csv", index=True)

swap for risk

In [521]:
results_return12=pd.DataFrame(columns=stocks)

In [522]:
for i in range(18):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["PFFD", "SRET", "QYLD","MMP","ED","IEP","EPD","HEP","SPH","SO","ETR","PPL","DUK","D","SUN","MPLX","OGE","EIX",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 

In [523]:
results_return12.to_csv("ALTYswapriskyear.csv", index=True)

# UYM

monthly

In [524]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [525]:
# call the function and generate a table with stock adjusted close prices
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
portfolio=combine_stocks(stocks)

                   LIN
Date                  
2022-04-19  316.985321
2022-04-20  325.175232
2022-04-21  323.182556
2022-04-22  308.586212
2022-04-25  312.840546
                   APD
Date                  
2022-04-19  247.907104
2022-04-20  247.082626
2022-04-21  249.466629
2022-04-22  241.301407
2022-04-25  241.142471
                  NEM
Date                 
2022-04-19  82.188332
2022-04-20  81.920509
2022-04-21  76.435341
2022-04-22  73.915932
2022-04-25  72.021416
                  FCX
Date                 
2022-04-19  50.352707
2022-04-20  49.616909
2022-04-21  44.694988
2022-04-22  41.672234
2022-04-25  41.413712
                  DOW
Date                 
2022-04-19  66.358154
2022-04-20  66.843094
2022-04-21  68.792770
2022-04-22  67.793190
2022-04-25  67.367630
                 CTVA
Date                 
2022-04-19  60.828083
2022-04-20  61.007168
2022-04-21  59.902828
2022-04-22  57.385735
2022-04-25  57.007675
                   ECL
Date                  
2022-04-19  180.

In [526]:
# call the function and generate a table with stock adjusted close prices
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
portfolio2=combine_stocks(stocks)

                   LIN
Date                  
2022-04-19  316.985321
2022-04-20  325.175232
2022-04-21  323.182556
2022-04-22  308.586212
2022-04-25  312.840546
                   APD
Date                  
2022-04-19  247.907104
2022-04-20  247.082626
2022-04-21  249.466629
2022-04-22  241.301407
2022-04-25  241.142471
                  NEM
Date                 
2022-04-19  82.188332
2022-04-20  81.920509
2022-04-21  76.435341
2022-04-22  73.915932
2022-04-25  72.021416
                  FCX
Date                 
2022-04-19  50.352707
2022-04-20  49.616909
2022-04-21  44.694988
2022-04-22  41.672234
2022-04-25  41.413712
                  DOW
Date                 
2022-04-19  66.358154
2022-04-20  66.843094
2022-04-21  68.792770
2022-04-22  67.793190
2022-04-25  67.367630
                 CTVA
Date                 
2022-04-19  60.828083
2022-04-20  61.007168
2022-04-21  59.902828
2022-04-22  57.385735
2022-04-25  57.007675
                   ECL
Date                  
2022-04-19  180.

delete for mean variance

In [527]:
results_return1=pd.DataFrame(columns=stocks)

In [529]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return1[results_return1.columns[i]]=np.array(result) 

In [530]:
results_return1

,LIN,APD,NEM,FCX,DOW,CTVA,ECL,NUE,DD,IFF,PPG,LYB,ALB,CF,MOS,CE,STLD,FMC,EMN,RS,RPM,AA,CLF,RGLD,OLN,HUN,X,VVV,CC,ASH,CMC,ESI,WLK,SMG,DNA,NGVT,MP,NEU
0,0.168756,0.168805,0.168880,0.168810,0.168836,0.168793,0.168789,0.169056,0.168761,0.168844,0.168849,0.168831,0.169308,0.168986,0.169429,0.168882,0.168960,0.168952,0.168776,0.168862,0.168801,0.169440,0.168762,0.168821,0.168952,0.168803,0.168789,0.168797,0.066682,0.168783,0.168803,0.168782,0.168839,0.169015,0.169308,0.169308,0.169027,0.168827
1,0.214020,0.214097,0.214215,0.214104,0.214145,0.214078,0.214071,0.214492,0.214027,0.214159,0.214166,0.214138,0.214889,0.214381,0.214967,0.214218,0.214341,0.214329,0.214052,0.214187,0.214090,0.214975,0.214029,0.214122,0.214329,0.214094,0.214071,0.214083,0.112660,0.214062,0.214094,0.214060,0.214150,0.214427,0.214889,0.214889,0.214446,0.214131
2,0.649268,0.649263,0.649255,0.649263,0.649260,0.649264,0.649265,0.649237,0.649268,0.649259,0.649259,0.649260,0.649209,0.649244,0.649537,0.649255,0.649247,0.649248,0.649266,0.649257,0.649264,0.649565,0.649268,0.649261,0.649248,0.649263,0.649265,0.649264,0.327377,0.649265,0.649263,0.649266,0.649260,0.649241,0.649209,0.649209,0.649240,0.649261


In [531]:
results_return1.to_csv("UYMdeletemeanmonth.csv", index=True)

swap for mean

In [532]:
results_return2=pd.DataFrame(columns=stocks)

In [533]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return2[results_return2.columns[i]]=np.array(result) 

In [534]:
results_return2.to_csv("UYMswapmeanmonth.csv", index=True)

delete for risk

In [535]:
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
results_return3=pd.DataFrame(columns=stocks)

In [536]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return3[results_return3.columns[i]]=np.array(result) 

In [537]:
results_return3.to_csv("UYMdeleteriskmonth.csv", index=True)

swap for risk

In [538]:
results_return4=pd.DataFrame(columns=stocks)

In [539]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return4[results_return4.columns[i]]=np.array(result) 

In [540]:
results_return4.to_csv("UYMswapriskmonth.csv", index=True)

quarterly

In [541]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [542]:
# call the function and generate a table with stock adjusted close prices
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
portfolio=combine_stocks(stocks)

                   LIN
Date                  
2021-12-20  327.795563
2021-12-21  330.345947
2021-12-22  332.787140
2021-12-23  336.577911
2021-12-27  340.795471
                   APD
Date                  
2021-12-20  289.731903
2021-12-21  290.458679
2021-12-22  291.843445
2021-12-23  294.121918
2021-12-27  296.812897
                  NEM
Date                 
2021-12-20  57.821400
2021-12-21  57.299423
2021-12-22  57.614582
2021-12-23  58.628990
2021-12-27  59.269157
                  FCX
Date                 
2021-12-20  36.968208
2021-12-21  38.706955
2021-12-22  39.655365
2021-12-23  40.633415
2021-12-27  41.483028
                  DOW
Date                 
2021-12-20  52.326550
2021-12-21  53.001293
2021-12-22  53.030628
2021-12-23  53.920502
2021-12-27  54.634354
                 CTVA
Date                 
2021-12-20  44.411385
2021-12-21  45.463135
2021-12-22  45.572277
2021-12-23  46.078300
2021-12-27  46.643860
                   ECL
Date                  
2021-12-20  222.

In [543]:
# call the function and generate a table with stock adjusted close prices
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
portfolio2=combine_stocks(stocks)

                   LIN
Date                  
2021-12-20  327.795563
2021-12-21  330.345947
2021-12-22  332.787140
2021-12-23  336.577911
2021-12-27  340.795471
                   APD
Date                  
2021-12-20  289.731934
2021-12-21  290.458649
2021-12-22  291.843445
2021-12-23  294.121918
2021-12-27  296.812897
                  NEM
Date                 
2021-12-20  57.821400
2021-12-21  57.299423
2021-12-22  57.614582
2021-12-23  58.628990
2021-12-27  59.269157
                  FCX
Date                 
2021-12-20  36.968208
2021-12-21  38.706959
2021-12-22  39.655365
2021-12-23  40.633419
2021-12-27  41.483032
                  DOW
Date                 
2021-12-20  52.326550
2021-12-21  53.001293
2021-12-22  53.030628
2021-12-23  53.920502
2021-12-27  54.634354
                 CTVA
Date                 
2021-12-20  44.411388
2021-12-21  45.463131
2021-12-22  45.572277
2021-12-23  46.078300
2021-12-27  46.643860
                   ECL
Date                  
2021-12-20  222.

In [544]:
results_return5=pd.DataFrame(columns=stocks)

In [545]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return5[results_return5.columns[i]]=np.array(result)

In [547]:
results_return5.to_csv("UYMdeletemeanquarter.csv", index=True)

In [548]:
# calculate the expected return using historical data
mu = mean_historical_return(portfolio2, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio2).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio2.columns)
for j in df:
    df[j]=portfolio2[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]

In [549]:
result

[0.4819004326569234, 0.175579435244034, 2.5749053813082323]

swap for mean variance

In [550]:
results_return6=pd.DataFrame(columns=stocks)

In [551]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 

In [552]:
results_return6.to_csv("UYMswapmeanquarter.csv", index=True)

delete for risk parity

In [553]:
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]


results_return7=pd.DataFrame(columns=stocks)

In [554]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 

In [555]:
results_return7.to_csv("UYMdeleteriskquarter.csv", index=True)

swap for risk

In [556]:
results_return8=pd.DataFrame(columns=stocks)

In [557]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 

In [558]:
results_return8.to_csv("UYMswapriskquarter.csv", index=True)

year

In [559]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [560]:
# call the function and generate a table with stock adjusted close prices
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']
portfolio=combine_stocks(stocks)

                   LIN
Date                  
2020-06-19  201.966614
2020-06-22  203.830444
2020-06-23  206.160294
2020-06-24  200.044510
2020-06-25  203.024719
                   APD
Date                  
2020-06-19  224.129379
2020-06-22  225.042969
2020-06-23  227.536377
2020-06-24  221.873886
2020-06-25  225.832901
                  NEM
Date                 
2020-06-19  53.485851
2020-06-22  54.707424
2020-06-23  55.355793
2020-06-24  54.885960
2020-06-25  55.055096
                  FCX
Date                 
2020-06-19  10.306809
2020-06-22  10.591475
2020-06-23  10.876138
2020-06-24  10.375523
2020-06-25  10.876138
                  DOW
Date                 
2020-06-19  37.902519
2020-06-22  37.984425
2020-06-23  37.629513
2020-06-24  34.990440
2020-06-25  36.382778
                 CTVA
Date                 
2020-06-19  26.379171
2020-06-22  26.816719
2020-06-23  26.661148
2020-06-24  25.348509
2020-06-25  25.474913
                   ECL
Date                  
2020-06-19  195.

In [561]:
# call the function and generate a table with stock adjusted close prices
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
portfolio2=combine_stocks(stocks)

                   LIN
Date                  
2020-06-19  201.966583
2020-06-22  203.830460
2020-06-23  206.160278
2020-06-24  200.044479
2020-06-25  203.024719
                   APD
Date                  
2020-06-19  224.129364
2020-06-22  225.042984
2020-06-23  227.536362
2020-06-24  221.873917
2020-06-25  225.832870
                  NEM
Date                 
2020-06-19  53.485855
2020-06-22  54.707420
2020-06-23  55.355793
2020-06-24  54.885956
2020-06-25  55.055096
                  FCX
Date                 
2020-06-19  10.306809
2020-06-22  10.591473
2020-06-23  10.876140
2020-06-24  10.375523
2020-06-25  10.876140
                  DOW
Date                 
2020-06-19  37.902527
2020-06-22  37.984425
2020-06-23  37.629513
2020-06-24  34.990440
2020-06-25  36.382786
                 CTVA
Date                 
2020-06-19  26.379171
2020-06-22  26.816719
2020-06-23  26.661146
2020-06-24  25.348507
2020-06-25  25.474909
                   ECL
Date                  
2020-06-19  195.

delete for mean variance

In [562]:
results_return9=pd.DataFrame(columns=stocks)

In [564]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result)

In [565]:
results_return9.to_csv("UYMdeletemeanyear.csv", index=True)

swap for mean variance

In [566]:
results_return10=pd.DataFrame(columns=stocks)

In [567]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 

In [568]:
results_return10.to_csv("UYMswapmeanyear.csv", index=True)

delete for risk

In [571]:
stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]
results_return11=pd.DataFrame(columns=stocks)

In [572]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result)

In [573]:
results_return11.to_csv("UYMdeleteriskyear.csv", index=True)

swap for risk

In [574]:
results_return12=pd.DataFrame(columns=stocks)

In [575]:
for i in range(38):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["LIN", "APD", "NEM","FCX","DOW","CTVA","ECL","NUE","DD","IFF","PPG","LYB","ALB","CF","MOS","CE","STLD","FMC","EMN", "RS", "RPM","AA","CLF","RGLD","OLN","HUN","X","VVV","CC","ASH","CMC","ESI","WLK","SMG","DNA","NGVT","MP","NEU",'^TNX']

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 

In [577]:
results_return12.to_csv("UYMswapriskyear.csv", index=True)

# AIA

monthly

In [578]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [579]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']
portfolio=combine_stocks(stocks)

               2330.TW
Date                  
2022-04-19  561.947449
2022-04-20  566.920410
2022-04-21  561.947449
2022-04-22  554.985291
2022-04-25  544.044678
               0700.HK
Date                  
2022-04-19  362.296082
2022-04-20  360.305450
2022-04-21  346.370972
2022-04-22  339.005615
2022-04-25  325.867401
               005930.KS
Date                    
2022-04-19  66890.984375
2022-04-20  66990.375000
2022-04-21  67288.554688
2022-04-22  66592.812500
2022-04-25  65897.062500
              9988.HK
Date                 
2022-04-19  91.500000
2022-04-20  90.699997
2022-04-21  87.900002
2022-04-22  86.650002
2022-04-25  81.849998
              1299.HK
Date                 
2022-04-19  77.276161
2022-04-20  78.509033
2022-04-21  78.213150
2022-04-22  77.177536
2022-04-25  73.972076
               3690.HK
Date                  
2022-04-19  146.000000
2022-04-20  144.899994
2022-04-21  137.800003
2022-04-22  141.399994
2022-04-25  138.600006
             0939.HK
Date         

               STLAX
Date                
2022-04-19  9.882961
2022-04-20  9.922611
2022-04-21  9.833397
2022-04-22  9.724359
2022-04-25  9.754096
            051915.KS
Date                 
2022-04-19     257000
2022-04-20     257000
2022-04-21     256000
2022-04-22     251000
2022-04-25     247000
               005385.KS
Date                    
2022-04-19  88700.000000
2022-04-20  89886.625000
2022-04-21  90776.585938
2022-04-22  90677.703125
2022-04-25  90183.281250
                 MCHI
Date                 
2022-04-19  51.172371
2022-04-20  49.621094
2022-04-21  47.920647
2022-04-22  48.218975
2022-04-25  47.294170
             ^TNX
Date             
2022-04-19  2.913
2022-04-20  2.840
2022-04-21  2.917
2022-04-22  2.906
2022-04-25  2.826


In [580]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]
portfolio2=combine_stocks(stocks)

               2330.TW
Date                  
2022-04-19  561.947449
2022-04-20  566.920410
2022-04-21  561.947449
2022-04-22  554.985291
2022-04-25  544.044678
               0700.HK
Date                  
2022-04-19  362.296082
2022-04-20  360.305450
2022-04-21  346.370972
2022-04-22  339.005615
2022-04-25  325.867401
               005930.KS
Date                    
2022-04-19  66890.984375
2022-04-20  66990.375000
2022-04-21  67288.554688
2022-04-22  66592.812500
2022-04-25  65897.062500
              9988.HK
Date                 
2022-04-19  91.500000
2022-04-20  90.699997
2022-04-21  87.900002
2022-04-22  86.650002
2022-04-25  81.849998
              1299.HK
Date                 
2022-04-19  77.276161
2022-04-20  78.509033
2022-04-21  78.213150
2022-04-22  77.177536
2022-04-25  73.972076
               3690.HK
Date                  
2022-04-19  146.000000
2022-04-20  144.899994
2022-04-21  137.800003
2022-04-22  141.399994
2022-04-25  138.600006
             0939.HK
Date         

               STLAX
Date                
2022-04-19  9.882961
2022-04-20  9.922612
2022-04-21  9.833398
2022-04-22  9.724359
2022-04-25  9.754096
            051915.KS
Date                 
2022-04-19     257000
2022-04-20     257000
2022-04-21     256000
2022-04-22     251000
2022-04-25     247000
               005385.KS
Date                    
2022-04-19  88700.000000
2022-04-20  89886.625000
2022-04-21  90776.585938
2022-04-22  90677.703125
2022-04-25  90183.281250
                 MCHI
Date                 
2022-04-19  51.172371
2022-04-20  49.621094
2022-04-21  47.920647
2022-04-22  48.218975
2022-04-25  47.294170


delete for mean

In [582]:
results_return1=pd.DataFrame(columns=stocks)

In [583]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return1[results_return1.columns[i]]=np.array(result) 

In [598]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
result

[0.08181108177418699, 0.06436025343334838, 0.8081242537065174]

In [584]:
results_return1.to_csv("AIAdeletemeanmonth.csv", index=True)

swap for mean variance

In [585]:
results_return2=pd.DataFrame(columns=stocks)

In [588]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return2[results_return2.columns[i]]=np.array(result) 

In [589]:
results_return2.to_csv("AIAswapmeanmonth.csv", index=True)

delete for risk

In [591]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]


results_return3=pd.DataFrame(columns=stocks)

In [593]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,30)
    sd=portfolio_sd(df,cleaned_weights,30)
    sharpe=sharpe_fun(df,cleaned_weights,30,0.0298)
    result=[returns,sd,sharpe]
    results_return3[results_return3.columns[i]]=np.array(result) 

In [594]:
results_return3.to_csv("AIAdeleteriskmonth.csv", index=True)

swap for risk

In [595]:
results_return4=pd.DataFrame(columns=stocks)

In [596]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,30)
    sd=portfolio_sd(df,cleaned_weights,30)
    sharpe=sharpe_fun(df,cleaned_weights,30,0.0298)
    result=[returns,sd,sharpe]
    results_return4[results_return4.columns[i]]=np.array(result) 

In [597]:
results_return4.to_csv("AIAswapriskmonth.csv", index=True)

quarterly

In [599]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [600]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']
portfolio=combine_stocks(stocks)

               2330.TW
Date                  
2021-12-20  591.837952
2021-12-21  590.848267
2021-12-22  593.817383
2021-12-23  599.755554
2021-12-24  597.776123
               0700.HK
Date                  
2021-12-20  427.987122
2021-12-21  438.736542
2021-12-22  440.926270
2021-12-23  459.638245
2021-12-24  458.642944
               005930.KS
Date                    
2021-12-20  75894.617188
2021-12-21  76878.984375
2021-12-22  78158.656250
2021-12-23  78650.843750
2021-12-24  79241.460938
               9988.HK
Date                  
2021-12-20  114.500000
2021-12-21  113.900002
2021-12-22  114.900002
2021-12-23  113.300003
2021-12-24  113.000000
              1299.HK
Date                 
2021-12-20  77.128220
2021-12-21  76.881645
2021-12-22  77.276161
2021-12-23  77.522736
2021-12-24  77.867935
               3690.HK
Date                  
2021-12-20  219.199997
2021-12-21  229.000000
2021-12-22  230.399994
2021-12-23  226.600006
2021-12-24  225.399994
             0939.HK
Date  

               005387.KS
Date                    
2021-12-20  94830.304688
2021-12-21  95495.445312
2021-12-22  96920.750000
2021-12-23  96920.750000
2021-12-24  99771.367188
                STLAX
Date                 
2021-12-20  10.569155
2021-12-21  10.650387
2021-12-22  10.695515
2021-12-23  10.722593
2021-12-27  10.776747
               051915.KS
Date                    
2021-12-20  288608.28125
2021-12-21  281885.31250
2021-12-22  275642.53125
2021-12-23  284286.37500
2021-12-24  284286.37500
               005385.KS
Date                    
2021-12-20  93762.007812
2021-12-21  94047.000000
2021-12-22  94426.984375
2021-12-23  94901.968750
2021-12-24  96896.906250
                 MCHI
Date                 
2021-12-20  60.360725
2021-12-21  62.190441
2021-12-22  61.971672
2021-12-23  62.379379
2021-12-27  62.170555
             ^TNX
Date             
2021-12-20  1.419
2021-12-21  1.487
2021-12-22  1.457
2021-12-23  1.493
2021-12-27  1.481


In [601]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]
portfolio2=combine_stocks(stocks)

               2330.TW
Date                  
2021-12-20  591.837952
2021-12-21  590.848267
2021-12-22  593.817383
2021-12-23  599.755554
2021-12-24  597.776123
               0700.HK
Date                  
2021-12-20  427.987122
2021-12-21  438.736542
2021-12-22  440.926270
2021-12-23  459.638245
2021-12-24  458.642944
               005930.KS
Date                    
2021-12-20  75894.617188
2021-12-21  76878.984375
2021-12-22  78158.656250
2021-12-23  78650.843750
2021-12-24  79241.460938
               9988.HK
Date                  
2021-12-20  114.500000
2021-12-21  113.900002
2021-12-22  114.900002
2021-12-23  113.300003
2021-12-24  113.000000
              1299.HK
Date                 
2021-12-20  77.128220
2021-12-21  76.881645
2021-12-22  77.276161
2021-12-23  77.522736
2021-12-24  77.867935
               3690.HK
Date                  
2021-12-20  219.199997
2021-12-21  229.000000
2021-12-22  230.399994
2021-12-23  226.600006
2021-12-24  225.399994
             0939.HK
Date  

               005387.KS
Date                    
2021-12-20  94830.304688
2021-12-21  95495.453125
2021-12-22  96920.757812
2021-12-23  96920.757812
2021-12-24  99771.359375
                STLAX
Date                 
2021-12-20  10.569155
2021-12-21  10.650387
2021-12-22  10.695516
2021-12-23  10.722592
2021-12-27  10.776747
               051915.KS
Date                    
2021-12-20  288608.28125
2021-12-21  281885.31250
2021-12-22  275642.53125
2021-12-23  284286.37500
2021-12-24  284286.37500
               005385.KS
Date                    
2021-12-20  93762.007812
2021-12-21  94047.000000
2021-12-22  94426.992188
2021-12-23  94901.968750
2021-12-24  96896.906250
                 MCHI
Date                 
2021-12-20  60.360725
2021-12-21  62.190441
2021-12-22  61.971672
2021-12-23  62.379379
2021-12-27  62.170555


delete for mean variance

In [602]:
results_return5=pd.DataFrame(columns=stocks)

In [603]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return5[results_return5.columns[i]]=np.array(result)

In [604]:
results_return5.to_csv("AIAdeletemeanquarter.csv", index=True)

In [606]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
result

[0.10200325314773567, 0.0788814392226906, 0.9153389423321537]

swap for mean variance

In [607]:
results_return6=pd.DataFrame(columns=stocks)

In [617]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 

In [618]:
results_return6.to_csv("AIAswapmeanquarter.csv", index=True)

In [612]:
results_return6

,2330.TW,0700.HK,005930.KS,9988.HK,1299.HK,3690.HK,0939.HK,9618.HK,0388.HK,2317.TW,2454.TW,9888.HK,2318.HK,D05.SI,000660.KS,1398.HK,9999.HK,005935.KS,O39.SI,1211.HK,3988.HK,U11.SI,2269.HK,035420.KS,3968.HK,006400.KS,051910.KS,1810.HK,2331.HK,005380.KS,0016.HK,0001.HK,0669.HK,2303.TW,0823.HK,2891.TW,035720.KS,2881.TW,2412.TW,1303.TW,1301.TW,000270.KS,055550.KS,2882.TW,2308.TW,105560.KS,0002.HK,005490.KS,068270.KS,2020.HK,0003.HK,005387.KS,STLAX,051915.KS,005385.KS,MCHI
0,0.122572,0.122619,0.122578,0.122775,0.122552,0.122456,0.123083,0.122738,0.122509,0.122810,0.122558,0.122487,0.122518,0.122690,0.122625,0.123472,0.122628,0.122577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.085738,0.085865,0.085752,0.086300,0.085673,0.085410,0.085819,0.086200,0.085558,0.085892,0.085688,0.085496,0.085588,0.085529,0.085625,0.086232,0.085895,0.085753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.082045,1.080987,1.081923,1.077351,1.082628,1.084836,1.086975,1.078167,1.083575,1.082863,1.082513,1.084105,1.083310,1.086059,1.084086,1.086283,1.080722,1.081914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [615]:
stock=stocks[0:18]+stocks[19:]
portfolio1=portfolio[stock]  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
results_return6[results_return6.columns[18]]=np.array(result) 

In [616]:
results_return6

,2330.TW,0700.HK,005930.KS,9988.HK,1299.HK,3690.HK,0939.HK,9618.HK,0388.HK,2317.TW,2454.TW,9888.HK,2318.HK,D05.SI,000660.KS,1398.HK,9999.HK,005935.KS,O39.SI,1211.HK,3988.HK,U11.SI,2269.HK,035420.KS,3968.HK,006400.KS,051910.KS,1810.HK,2331.HK,005380.KS,0016.HK,0001.HK,0669.HK,2303.TW,0823.HK,2891.TW,035720.KS,2881.TW,2412.TW,1303.TW,1301.TW,000270.KS,055550.KS,2882.TW,2308.TW,105560.KS,0002.HK,005490.KS,068270.KS,2020.HK,0003.HK,005387.KS,STLAX,051915.KS,005385.KS,MCHI
0,0.122572,0.122619,0.122578,0.122775,0.122552,0.122456,0.123083,0.122738,0.122509,0.122810,0.122558,0.122487,0.122518,0.122690,0.122625,0.123472,0.122628,0.122577,0.121488,0.122520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.085738,0.085865,0.085752,0.086300,0.085673,0.085410,0.085819,0.086200,0.085558,0.085892,0.085688,0.085496,0.085588,0.085529,0.085625,0.086232,0.085895,0.085753,0.084421,0.085589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.082045,1.080987,1.081923,1.077351,1.082628,1.084836,1.086975,1.078167,1.083575,1.082863,1.082513,1.084105,1.083310,1.086059,1.084086,1.086283,1.080722,1.081914,1.086082,1.083315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


delete for risk parity

In [619]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]


results_return7=pd.DataFrame(columns=stocks)

In [620]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 

In [621]:
results_return7.to_csv("ALTYdeleteriskquarter.csv", index=True)

swap for risk parity

In [622]:
results_return8=pd.DataFrame(columns=stocks)

In [623]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 

In [624]:
results_return8.to_csv("AIAswapriskquarter.csv", index=True)

yearly

In [625]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [626]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']
portfolio=combine_stocks(stocks)

               2330.TW
Date                  
2020-06-19  302.746521
2020-06-22  300.339935
2020-06-23  303.227814
2020-06-24  305.634369
2020-06-29  300.339935
               0700.HK
Date                  
2020-06-19  456.998932
2020-06-22  470.696991
2020-06-23  493.725616
2020-06-24  487.174347
2020-06-26  481.417236
               005930.KS
Date                    
2020-06-19  49471.664062
2020-06-22  48629.992188
2020-06-23  48068.875000
2020-06-24  49471.664062
2020-06-25  48536.468750
               9988.HK
Date                  
2020-06-19  219.000000
2020-06-22  213.000000
2020-06-23  220.000000
2020-06-24  221.399994
2020-06-26  214.600006
              1299.HK
Date                 
2020-06-19  69.518761
2020-06-22  70.777443
2020-06-23  72.520256
2020-06-24  73.198013
2020-06-26  71.745674
               3690.HK
Date                  
2020-06-19  174.000000
2020-06-22  170.500000
2020-06-23  176.800003
2020-06-24  178.399994
2020-06-26  174.699997
             0939.HK
Date  

               STLAX
Date                
2020-06-19  8.926378
2020-06-22  8.958867
2020-06-23  8.983234
2020-06-24  8.885768
2020-06-25  8.926378
                051915.KS
Date                     
2020-06-19  258450.546875
2020-06-22  257983.218750
2020-06-23  257983.218750
2020-06-24  255646.375000
2020-06-25  245364.453125
               005385.KS
Date                    
2020-06-19  52507.046875
2020-06-22  51051.046875
2020-06-23  51506.046875
2020-06-24  52780.046875
2020-06-25  51142.046875
                 MCHI
Date                 
2020-06-19  63.621586
2020-06-22  64.491653
2020-06-23  65.420364
2020-06-24  64.814262
2020-06-25  64.579628
             ^TNX
Date             
2020-06-19  0.697
2020-06-22  0.704
2020-06-23  0.709
2020-06-24  0.684
2020-06-25  0.674


In [627]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]
portfolio2=combine_stocks(stocks)

               2330.TW
Date                  
2020-06-19  302.746521
2020-06-22  300.339935
2020-06-23  303.227783
2020-06-24  305.634369
2020-06-29  300.339935
               0700.HK
Date                  
2020-06-19  456.998932
2020-06-22  470.696991
2020-06-23  493.725616
2020-06-24  487.174347
2020-06-26  481.417236
               005930.KS
Date                    
2020-06-19  49471.660156
2020-06-22  48629.988281
2020-06-23  48068.878906
2020-06-24  49471.660156
2020-06-25  48536.472656
               9988.HK
Date                  
2020-06-19  219.000000
2020-06-22  213.000000
2020-06-23  220.000000
2020-06-24  221.399994
2020-06-26  214.600006
              1299.HK
Date                 
2020-06-19  69.518753
2020-06-22  70.777443
2020-06-23  72.520264
2020-06-24  73.198013
2020-06-26  71.745674
               3690.HK
Date                  
2020-06-19  174.000000
2020-06-22  170.500000
2020-06-23  176.800003
2020-06-24  178.399994
2020-06-26  174.699997
             0939.HK
Date  

ReadTimeout: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out. (read timeout=30)

delete for mean variance

In [629]:
results_return9=pd.DataFrame(columns=stocks)

In [636]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result)

In [637]:
results_return9.to_csv("AIAdeletemeanyear.csv", index=True)

swap for mean variance

In [632]:
results_return10=pd.DataFrame(columns=stocks)

In [634]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 

In [635]:
results_return10.to_csv("AIAswapmeanyear.csv", index=True)

delete for risk

In [638]:
stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]
results_return11=pd.DataFrame(columns=stocks)

In [639]:
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result)

In [640]:
results_return11.to_csv("AIAdeleteriskyear.csv", index=True)

swap for risk

In [641]:
results_return12=pd.DataFrame(columns=stocks)
for i in range(56):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["2330.TW", "0700.HK", "005930.KS","9988.HK","1299.HK","3690.HK","0939.HK","9618.HK","0388.HK","2317.TW","2454.TW","9888.HK","2318.HK","D05.SI","000660.KS","1398.HK","9999.HK","005935.KS","O39.SI","1211.HK","3988.HK","U11.SI","2269.HK","035420.KS","3968.HK","006400.KS","051910.KS","1810.HK","2331.HK","005380.KS","0016.HK","0001.HK","0669.HK","2303.TW","0823.HK","2891.TW","035720.KS","2881.TW","2412.TW","1303.TW","1301.TW","000270.KS","055550.KS","2882.TW","2308.TW","105560.KS","0002.HK","005490.KS","068270.KS","2020.HK","0003.HK","005387.KS","STLAX","051915.KS","005385.KS","MCHI",'^TNX']

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 

In [642]:
results_return12.to_csv("AIAswapriskyear.csv", index=True)

# AIQ

monthly

In [643]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [644]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
portfolio=combine_stocks(stocks)

                 BABA
Date                 
2022-04-19  93.500000
2022-04-20  89.410004
2022-04-21  85.989998
2022-04-22  86.489998
2022-04-25  85.839996
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564087
2022-04-22  272.814178
2022-04-25  279.474487
                   IBM
Date                  
2022-04-19  126.014412
2022-04-20  134.961792
2022-04-21  136.454636
2022-04-22  134.893478
2022-04-25  135.722855
                 GOOGL
Date                  
2022-04-19  130.009003
2022-04-20  128.039993
2022-04-21  124.814499
2022-04-22  119.635498
2022-04-25  123.073997
                  AAPL
Date                  
2022-04-19  166.922546
2022-04-20  166.753021
2022-04-21  165.945343
2022-04-22  161.328537
2022-04-25  162.415436
                   ACN
Date                  
2022-04-19  319.433777
2022-04-20  322.522797
2022-04-21  321.167603
2022-04-22  309.698395
2022-04-25  315.697083
                 ORCL
Date               

                  BAH
Date                 
2022-04-19  86.534477
2022-04-20  88.980568
2022-04-21  89.030098
2022-04-22  87.168282
2022-04-25  86.524574
                  DXC
Date                 
2022-04-19  31.410000
2022-04-20  31.889999
2022-04-21  30.709999
2022-04-22  29.430000
2022-04-25  30.139999
                  OKTA
Date                  
2022-04-19  149.050003
2022-04-20  144.759995
2022-04-21  138.770004
2022-04-22  136.330002
2022-04-25  138.399994
                  TDC
Date                 
2022-04-19  45.430000
2022-04-20  45.779999
2022-04-21  45.139999
2022-04-22  43.740002
2022-04-25  43.650002
               3443.TW
Date                  
2022-04-19  465.174927
2022-04-20  463.693451
2022-04-21  466.656372
2022-04-22  438.508850
2022-04-25  400.978790
            CCCS
Date            
2022-04-19  9.67
2022-04-20  9.61
2022-04-21  9.34
2022-04-22  9.27
2022-04-25  9.70
                  HUBB
Date                  
2022-04-19  187.559250
2022-04-20  189.925873
2022-

In [648]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]
portfolio2=combine_stocks(stocks)

                 BABA
Date                 
2022-04-19  93.500000
2022-04-20  89.410004
2022-04-21  85.989998
2022-04-22  86.489998
2022-04-25  85.839996
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564056
2022-04-22  272.814178
2022-04-25  279.474487
                   IBM
Date                  
2022-04-19  126.014420
2022-04-20  134.961792
2022-04-21  136.454651
2022-04-22  134.893478
2022-04-25  135.722855
                 GOOGL
Date                  
2022-04-19  130.009003
2022-04-20  128.039993
2022-04-21  124.814499
2022-04-22  119.635498
2022-04-25  123.073997
                  AAPL
Date                  
2022-04-19  166.922531
2022-04-20  166.753021
2022-04-21  165.945328
2022-04-22  161.328537
2022-04-25  162.415436
                   ACN
Date                  
2022-04-19  319.433777
2022-04-20  322.522797
2022-04-21  321.167603
2022-04-22  309.698395
2022-04-25  315.697083
                 ORCL
Date               

                  BAH
Date                 
2022-04-19  86.534477
2022-04-20  88.980568
2022-04-21  89.030098
2022-04-22  87.168282
2022-04-25  86.524574
                  DXC
Date                 
2022-04-19  31.410000
2022-04-20  31.889999
2022-04-21  30.709999
2022-04-22  29.430000
2022-04-25  30.139999
                  OKTA
Date                  
2022-04-19  149.050003
2022-04-20  144.759995
2022-04-21  138.770004
2022-04-22  136.330002
2022-04-25  138.399994
                  TDC
Date                 
2022-04-19  45.430000
2022-04-20  45.779999
2022-04-21  45.139999
2022-04-22  43.740002
2022-04-25  43.650002
               3443.TW
Date                  
2022-04-19  465.174927
2022-04-20  463.693451
2022-04-21  466.656372
2022-04-22  438.508850
2022-04-25  400.978790
            CCCS
Date            
2022-04-19  9.67
2022-04-20  9.61
2022-04-21  9.34
2022-04-22  9.27
2022-04-25  9.70
                  HUBB
Date                  
2022-04-19  187.559250
2022-04-20  189.925873
2022-

delete for mean variance

In [649]:
results_return1=pd.DataFrame(columns=stocks)

In [650]:
len(stocks)

81

In [651]:
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]

    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return1[results_return1.columns[i]]=np.array(result) 

In [652]:
results_return1.to_csv("AIQdeletemeanmonth.csv", index=True)

In [653]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
result

[0.11987008238061353, 0.08097713839329142, 1.112290260779028]

swap for mean variance

In [654]:
results_return2=pd.DataFrame(columns=stocks)

In [656]:
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return2[results_return2.columns[i]]=np.array(result) 

In [657]:
results_return2.to_csv("AIQswapmeanmonth.csv", index=True)

delete for risk parity

In [658]:
results_return3=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return3[results_return3.columns[i]]=np.array(result) 

In [659]:
results_return3.to_csv("AIQdeleteriskmonth.csv", index=True)

swap for risk parity

In [660]:
results_return4=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return4[results_return4.columns[i]]=np.array(result) 

In [661]:
results_return4.to_csv("AIQswapriskmonth.csv", index=True)

quarterly

In [662]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [663]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
portfolio=combine_stocks(stocks)

                  BABA
Date                  
2021-12-20  115.000000
2021-12-21  122.980003
2021-12-22  117.809998
2021-12-23  118.660004
2021-12-27  116.589996
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165527
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   IBM
Date                  
2021-12-20  122.499596
2021-12-21  124.341042
2021-12-22  125.093033
2021-12-23  125.941460
2021-12-27  126.895927
                 GOOGL
Date                  
2021-12-20  141.606995
2021-12-21  143.472504
2021-12-22  146.414993
2021-12-23  146.916504
2021-12-27  147.906494
                  AAPL
Date                  
2021-12-20  169.050461
2021-12-21  172.277100
2021-12-22  174.916183
2021-12-23  175.553543
2021-12-27  179.586868
                   ACN
Date                  
2021-12-20  382.280334
2021-12-21  389.325836
2021-12-22  400.027832
2021-12-23  399.651306
2021-12-27  411.562225
                 ORCL
Date        

                 SMCI
Date                 
2021-12-20  41.279999
2021-12-21  42.630001
2021-12-22  42.590000
2021-12-23  42.720001
2021-12-27  44.759998
                  BAH
Date                 
2021-12-20  79.946754
2021-12-21  81.020050
2021-12-22  81.128372
2021-12-23  81.729027
2021-12-27  82.713715
                  DXC
Date                 
2021-12-20  29.980000
2021-12-21  31.350000
2021-12-22  31.959999
2021-12-23  32.570000
2021-12-27  32.790001
                  OKTA
Date                  
2021-12-20  218.910004
2021-12-21  228.380005
2021-12-22  228.339996
2021-12-23  228.309998
2021-12-27  229.410004
                  TDC
Date                 
2021-12-20  42.250000
2021-12-21  43.119999
2021-12-22  43.080002
2021-12-23  44.110001
2021-12-27  44.439999
               3443.TW
Date                  
2021-12-20  593.567139
2021-12-21  604.431091
2021-12-22  598.505310
2021-12-23  595.542419
2021-12-24  596.530029
             CCCS
Date             
2021-12-20  11.06
2021-12-

In [664]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]
portfolio2=combine_stocks(stocks)

                  BABA
Date                  
2021-12-20  115.000000
2021-12-21  122.980003
2021-12-22  117.809998
2021-12-23  118.660004
2021-12-27  116.589996
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   IBM
Date                  
2021-12-20  122.499580
2021-12-21  124.341034
2021-12-22  125.093040
2021-12-23  125.941460
2021-12-27  126.895920
                 GOOGL
Date                  
2021-12-20  141.606995
2021-12-21  143.472504
2021-12-22  146.414993
2021-12-23  146.916504
2021-12-27  147.906494
                  AAPL
Date                  
2021-12-20  169.050461
2021-12-21  172.277100
2021-12-22  174.916183
2021-12-23  175.553558
2021-12-27  179.586868
                   ACN
Date                  
2021-12-20  382.280334
2021-12-21  389.325836
2021-12-22  400.027832
2021-12-23  399.651306
2021-12-27  411.562225
                 ORCL
Date        

                 SMCI
Date                 
2021-12-20  41.279999
2021-12-21  42.630001
2021-12-22  42.590000
2021-12-23  42.720001
2021-12-27  44.759998
                  BAH
Date                 
2021-12-20  79.946739
2021-12-21  81.020050
2021-12-22  81.128372
2021-12-23  81.729027
2021-12-27  82.713715
                  DXC
Date                 
2021-12-20  29.980000
2021-12-21  31.350000
2021-12-22  31.959999
2021-12-23  32.570000
2021-12-27  32.790001
                  OKTA
Date                  
2021-12-20  218.910004
2021-12-21  228.380005
2021-12-22  228.339996
2021-12-23  228.309998
2021-12-27  229.410004
                  TDC
Date                 
2021-12-20  42.250000
2021-12-21  43.119999
2021-12-22  43.080002
2021-12-23  44.110001
2021-12-27  44.439999
               3443.TW
Date                  
2021-12-20  593.567139
2021-12-21  604.431091
2021-12-22  598.505310
2021-12-23  595.542419
2021-12-24  596.530029
             CCCS
Date             
2021-12-20  11.06
2021-12-

delete for mean variance

In [665]:
results_return5=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return5[results_return5.columns[i]]=np.array(result)

In [670]:
results_return5.to_csv("AIQdeletemeanquarter.csv", index=True)

In [667]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
result

[0.19584622722126546, 0.1128363500817522, 1.4715668053863993]

swap for mean variance

In [668]:
results_return6=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 

In [669]:
results_return6.to_csv("AIQswapmeanquarter.csv", index=True)

delete for risk parity

In [672]:
results_return7=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 

In [673]:
results_return7.to_csv("AIQdeleteriskquarter.csv", index=True)

swap for risk

In [674]:
results_return8=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 

In [675]:
results_return8.to_csv("AIQswapriskquarter.csv", index=True)

yearly

In [676]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [677]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']
portfolio=combine_stocks(stocks)

                  BABA
Date                  
2020-06-19  220.639999
2020-06-22  221.410004
2020-06-23  228.750000
2020-06-24  226.220001
2020-06-25  222.160004
                  MSFT
Date                  
2020-06-19  191.307068
2020-06-22  196.620346
2020-06-23  197.933929
2020-06-24  193.944092
2020-06-25  196.394836
                   IBM
Date                  
2020-06-19  104.569046
2020-06-22  103.373680
2020-06-23  101.956322
2020-06-24   99.403351
2020-06-25  101.631851
                GOOGL
Date                 
2020-06-19  71.232002
2020-06-22  72.532997
2020-06-23  73.198997
2020-06-24  71.635002
2020-06-25  72.055000
                 AAPL
Date                 
2020-06-19  86.231705
2020-06-22  88.487854
2020-06-23  90.376617
2020-06-24  88.781281
2020-06-25  89.959900
                   ACN
Date                  
2020-06-19  195.796249
2020-06-22  196.816238
2020-06-23  201.479233
2020-06-24  196.077942
2020-06-25  211.116043
                 ORCL
Date                 
2020

                  BAH
Date                 
2020-06-19  76.660202
2020-06-22  76.775497
2020-06-23  76.871559
2020-06-24  74.171806
2020-06-25  74.488838
              DXC
Date             
2020-06-19  15.43
2020-06-22  15.49
2020-06-23  16.24
2020-06-24  15.04
2020-06-25  15.91
                  OKTA
Date                  
2020-06-19  198.190002
2020-06-22  199.190002
2020-06-23  195.619995
2020-06-24  195.820007
2020-06-25  204.220001
                  TDC
Date                 
2020-06-19  20.330000
2020-06-22  20.219999
2020-06-23  20.410000
2020-06-24  20.070000
2020-06-25  20.469999
               3443.TW
Date                  
2020-06-19  211.254593
2020-06-22  214.661911
2020-06-23  236.079437
2020-06-24  236.079437
2020-06-29  253.116104
             CCCS
Date             
2020-10-05  11.78
2020-10-06  11.78
2020-10-07  11.60
2020-10-08  11.37
2020-10-09  11.40
                  HUBB
Date                  
2020-06-19  120.713150
2020-06-22  120.340111
2020-06-23  120.005325
202

In [678]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]
portfolio2=combine_stocks(stocks)

                  BABA
Date                  
2020-06-19  220.639999
2020-06-22  221.410004
2020-06-23  228.750000
2020-06-24  226.220001
2020-06-25  222.160004
                  MSFT
Date                  
2020-06-19  191.307037
2020-06-22  196.620346
2020-06-23  197.933929
2020-06-24  193.944061
2020-06-25  196.394867
                   IBM
Date                  
2020-06-19  104.569054
2020-06-22  103.373672
2020-06-23  101.956322
2020-06-24   99.403358
2020-06-25  101.631844
                GOOGL
Date                 
2020-06-19  71.232002
2020-06-22  72.532997
2020-06-23  73.198997
2020-06-24  71.635002
2020-06-25  72.055000
                 AAPL
Date                 
2020-06-19  86.231697
2020-06-22  88.487846
2020-06-23  90.376610
2020-06-24  88.781281
2020-06-25  89.959900
                   ACN
Date                  
2020-06-19  195.796219
2020-06-22  196.816269
2020-06-23  201.479233
2020-06-24  196.077942
2020-06-25  211.116043
                 ORCL
Date                 
2020

                  BAH
Date                 
2020-06-19  76.660187
2020-06-22  76.775482
2020-06-23  76.871574
2020-06-24  74.171799
2020-06-25  74.488838
              DXC
Date             
2020-06-19  15.43
2020-06-22  15.49
2020-06-23  16.24
2020-06-24  15.04
2020-06-25  15.91
                  OKTA
Date                  
2020-06-19  198.190002
2020-06-22  199.190002
2020-06-23  195.619995
2020-06-24  195.820007
2020-06-25  204.220001
                  TDC
Date                 
2020-06-19  20.330000
2020-06-22  20.219999
2020-06-23  20.410000
2020-06-24  20.070000
2020-06-25  20.469999
               3443.TW
Date                  
2020-06-19  211.254593
2020-06-22  214.661911
2020-06-23  236.079437
2020-06-24  236.079437
2020-06-29  253.116089
             CCCS
Date             
2020-10-05  11.78
2020-10-06  11.78
2020-10-07  11.60
2020-10-08  11.37
2020-10-09  11.40
                  HUBB
Date                  
2020-06-19  120.713150
2020-06-22  120.340103
2020-06-23  120.005333
202

delete for mean variance

In [681]:
results_return9=pd.DataFrame(columns=stocks)

In [682]:
for i in range(81):
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result)

In [683]:
results_return9.to_csv("AIQdeletemeanyear.csv", index=True)

swap for mean variance

In [684]:
results_return10=pd.DataFrame(columns=stocks)

In [685]:
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 

In [686]:
results_return10.to_csv("AIQswapmeanyear.csv", index=True)

In [687]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,253)
sd=portfolio_sd(df,cleaned_weights,253)
sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
result=[returns,sd,sharpe]
result

[0.7285764481637943, 0.39890110259548484, 1.7517536141593602]

delete for risk

In [688]:
stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]

results_return11=pd.DataFrame(columns=stocks)

In [689]:
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result)
results_return11.to_csv("AIQdeleteriskyear.csv", index=True)

swap for risk parity

In [691]:
results_return12=pd.DataFrame(columns=stocks)
for i in range(81):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["BABA", "MSFT", "IBM","GOOGL","AAPL","ACN","ORCL","3690.HK","CRM","0700.HK","AMZN","CSCO","ADBE","TSLA","QCOM","INTC","005930.KS","NVDA","NOW","SIE.DE","META","NFLX","000660.KS","TRI","SNPS","UBER","CDNS","SHOP","BIDU","NXPL","IFX.DE","6954.T","TWTR","STM","WDAY","EBAY","DDOG","EXPN.L","WKL.AS","ERIC","ZS","ROK","TTD","MNDT","SNAP","6502.T","HPE","NLSN","TWLO","STX","ZBRA","SPLK","SMCI","BAH","DXC","OKTA","TDC","3443.TW","CCCS","HUBB","ABMD","G","2395.TW","6701.T","2353.TW","CAN","VRNT","AVV.L","AI","3661.TW","VNET","AMP","TEMN.SW","VSAT","PATH","ZEN","SMAR","PEGA","WIX","AMBA","UPST",'^TNX']

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 
results_return12.to_csv("AIQswapriskyear.csv", index=True)

# BATT

monthly

In [692]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [693]:
# call the function and generate a table with stock adjusted close prices
stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']
portfolio=combine_stocks(stocks)

             300750.SZ
Date                  
2022-04-19  440.260010
2022-04-20  407.000000
2022-04-21  409.109985
2022-04-22  415.230011
2022-04-25  390.100006
               1211.HK
Date                  
2022-04-19  239.906189
2022-04-20  238.506744
2022-04-21  231.109619
2022-04-22  231.509476
2022-04-25  219.314240
                  BHP
Date                 
2022-04-19  69.455841
2022-04-20  68.599464
2022-04-21  63.746655
2022-04-22  60.651203
2022-04-25  58.786797
                   TSLA
Date                   
2022-04-19  1028.150024
2022-04-20   977.200012
2022-04-21  1008.780029
2022-04-22  1005.049988
2022-04-25   998.020020
                GLEN.L
Date                  
2022-04-19  524.813416
2022-04-20  508.449829
2022-04-21  489.700012
2022-04-22  476.200012
2022-04-26  464.799988
                   LI
Date                 
2022-04-19  25.340000
2022-04-20  23.490000
2022-04-21  22.469999
2022-04-22  22.330000
2022-04-25  22.309999
                  NIO
Date               

              6121.TWO
Date                  
2022-04-19  273.364349
2022-04-20  270.666656
2022-04-21  272.914734
2022-04-22  271.116272
2022-04-25  263.922485
            LICY
Date            
2022-04-19  7.22
2022-04-20  6.92
2022-04-21  6.74
2022-04-22  6.51
2022-04-25  6.92
            020150.KS
Date                 
2022-04-19      90500
2022-04-20      89600
2022-04-21      90400
2022-04-22      89100
2022-04-25      83200
             ZEV
Date            
2022-04-19  4.75
2022-04-20  4.64
2022-04-21  4.40
2022-04-22  4.30
2022-04-25  4.50
                ANTM.JK
Date                   
2022-04-19  2805.840332
2022-04-20  2776.305176
2022-04-21  2746.770020
2022-04-22  2697.544678
2022-04-25  2579.404053
             FREY
Date             
2022-04-19  10.78
2022-04-20  10.34
2022-04-21   9.78
2022-04-22   9.45
2022-04-25   9.67
              WBX
Date             
2022-04-19  11.65
2022-04-20  11.61
2022-04-21  11.69
2022-04-22  11.82
2022-04-25  12.25
            6674.T
Date    

In [694]:
# call the function and generate a table with stock adjusted close prices
stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]
portfolio2=combine_stocks(stocks)

             300750.SZ
Date                  
2022-04-19  440.260010
2022-04-20  407.000000
2022-04-21  409.109985
2022-04-22  415.230011
2022-04-25  390.100006
               1211.HK
Date                  
2022-04-19  239.906189
2022-04-20  238.506744
2022-04-21  231.109619
2022-04-22  231.509476
2022-04-25  219.314240
                  BHP
Date                 
2022-04-19  69.455841
2022-04-20  68.599464
2022-04-21  63.746655
2022-04-22  60.651203
2022-04-25  58.786797
                   TSLA
Date                   
2022-04-19  1028.150024
2022-04-20   977.200012
2022-04-21  1008.780029
2022-04-22  1005.049988
2022-04-25   998.020020
                GLEN.L
Date                  
2022-04-19  524.813416
2022-04-20  508.449829
2022-04-21  489.700012
2022-04-22  476.200012
2022-04-26  464.799988
                   LI
Date                 
2022-04-19  25.340000
2022-04-20  23.490000
2022-04-21  22.469999
2022-04-22  22.330000
2022-04-25  22.309999
                  NIO
Date               

              6121.TWO
Date                  
2022-04-19  273.364349
2022-04-20  270.666656
2022-04-21  272.914734
2022-04-22  271.116272
2022-04-25  263.922485
            LICY
Date            
2022-04-19  7.22
2022-04-20  6.92
2022-04-21  6.74
2022-04-22  6.51
2022-04-25  6.92
            020150.KS
Date                 
2022-04-19      90500
2022-04-20      89600
2022-04-21      90400
2022-04-22      89100
2022-04-25      83200
             ZEV
Date            
2022-04-19  4.75
2022-04-20  4.64
2022-04-21  4.40
2022-04-22  4.30
2022-04-25  4.50
                ANTM.JK
Date                   
2022-04-19  2805.840332
2022-04-20  2776.305176
2022-04-21  2746.770020
2022-04-22  2697.544678
2022-04-25  2579.404053
             FREY
Date             
2022-04-19  10.78
2022-04-20  10.34
2022-04-21   9.78
2022-04-22   9.45
2022-04-25   9.67
              WBX
Date             
2022-04-19  11.65
2022-04-20  11.61
2022-04-21  11.69
2022-04-22  11.82
2022-04-25  12.25
            6674.T
Date    

delete for mean variance

In [695]:
len(stocks)

85

In [696]:
results_return1=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return1[results_return1.columns[i]]=np.array(result) 
results_return1.to_csv("BATTdeletemeanmonth.csv", index=True)

swap for mean variance

In [697]:
results_return2=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    # call the function and generate a table with stock adjusted close prices
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return2[results_return2.columns[i]]=np.array(result) 

In [698]:
results_return2.to_csv("ALTYswapmeanmonth.csv", index=True)

In [699]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
result

[0.13704489041023507, 0.11657165611322688, 0.9199911366624778]

delete for risk parity

In [700]:
results_return3=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]


    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return3[results_return3.columns[i]]=np.array(result) 
results_return3.to_csv("BATTdeleteriskmonth.csv", index=True)

swap for risk parity

In [703]:
results_return4=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']


    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,30)
    sd=portfolio_sd(df,cleaned_weights,30)
    sharpe=sharpe_fun(df,cleaned_weights,30,0.0298)
    result=[returns,sd,sharpe]
    results_return4[results_return4.columns[i]]=np.array(result) 
results_return4.to_csv("BATTswapriskmonth.csv", index=True)

quarterly

In [704]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [705]:
# call the function and generate a table with stock adjusted close prices
stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']
portfolio=combine_stocks(stocks)

             300750.SZ
Date                  
2021-12-20  610.210022
2021-12-21  604.070007
2021-12-22  610.239990
2021-12-23  622.070007
2021-12-24  576.799988
               1211.HK
Date                  
2021-12-20  251.501663
2021-12-21  255.899933
2021-12-22  264.896423
2021-12-23  266.095947
2021-12-24  261.897583
                  BHP
Date                 
2021-12-20  51.935772
2021-12-21  53.264942
2021-12-22  52.854595
2021-12-23  53.452274
2021-12-27  53.782337
                   TSLA
Date                   
2021-12-20   899.940002
2021-12-21   938.530029
2021-12-22  1008.869995
2021-12-23  1067.000000
2021-12-27  1093.939941
                GLEN.L
Date                  
2021-12-20  353.582794
2021-12-21  359.706909
2021-12-22  365.831055
2021-12-23  368.427673
2021-12-24  365.488098
                   LI
Date                 
2021-12-20  28.250000
2021-12-21  29.969999
2021-12-22  30.400000
2021-12-23  30.420000
2021-12-27  29.900000
                  NIO
Date               

             2362.HK
Date                
2021-12-20  1.137286
2021-12-21  1.177190
2021-12-22  1.197143
2021-12-23  1.257000
2021-12-24  1.276952
               FSF.DU
Date                 
2021-12-20  48.099998
2021-12-21  47.549999
2021-12-22  49.599998
2021-12-23  48.549999
2021-12-27  49.049999
              6121.TWO
Date                  
2021-12-20  287.751953
2021-12-21  287.751953
2021-12-22  287.751953
2021-12-23  293.147278
2021-12-24  293.147278
             LICY
Date             
2021-12-20   9.45
2021-12-21   9.70
2021-12-22   9.92
2021-12-23  10.05
2021-12-27   9.89
                020150.KS
Date                     
2021-12-20  121718.460938
2021-12-21  120221.921875
2021-12-22  125210.382812
2021-12-23  126208.078125
2021-12-24  126208.078125
             ZEV
Date            
2021-12-20  5.71
2021-12-21  5.82
2021-12-22  5.83
2021-12-23  6.79
2021-12-27  6.79
                ANTM.JK
Date                   
2021-12-20  2185.601807
2021-12-21  2224.982178
2021-12-22  221

In [706]:
# call the function and generate a table with stock adjusted close prices
stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]
portfolio2=combine_stocks(stocks)

             300750.SZ
Date                  
2021-12-20  610.210022
2021-12-21  604.070007
2021-12-22  610.239990
2021-12-23  622.070007
2021-12-24  576.799988
               1211.HK
Date                  
2021-12-20  251.501663
2021-12-21  255.899933
2021-12-22  264.896423
2021-12-23  266.095947
2021-12-24  261.897583
                  BHP
Date                 
2021-12-20  51.935772
2021-12-21  53.264942
2021-12-22  52.854595
2021-12-23  53.452274
2021-12-27  53.782337
                   TSLA
Date                   
2021-12-20   899.940002
2021-12-21   938.530029
2021-12-22  1008.869995
2021-12-23  1067.000000
2021-12-27  1093.939941
                GLEN.L
Date                  
2021-12-20  353.582794
2021-12-21  359.706909
2021-12-22  365.831055
2021-12-23  368.427673
2021-12-24  365.488098
                   LI
Date                 
2021-12-20  28.250000
2021-12-21  29.969999
2021-12-22  30.400000
2021-12-23  30.420000
2021-12-27  29.900000
                  NIO
Date               

             2362.HK
Date                
2021-12-20  1.137286
2021-12-21  1.177190
2021-12-22  1.197143
2021-12-23  1.257000
2021-12-24  1.276952
               FSF.DU
Date                 
2021-12-20  48.099998
2021-12-21  47.549999
2021-12-22  49.599998
2021-12-23  48.549999
2021-12-27  49.049999
              6121.TWO
Date                  
2021-12-20  287.751953
2021-12-21  287.751953
2021-12-22  287.751953
2021-12-23  293.147278
2021-12-24  293.147278
             LICY
Date             
2021-12-20   9.45
2021-12-21   9.70
2021-12-22   9.92
2021-12-23  10.05
2021-12-27   9.89
                020150.KS
Date                     
2021-12-20  121718.460938
2021-12-21  120221.921875
2021-12-22  125210.382812
2021-12-23  126208.078125
2021-12-24  126208.078125
             ZEV
Date            
2021-12-20  5.71
2021-12-21  5.82
2021-12-22  5.83
2021-12-23  6.79
2021-12-27  6.79
                ANTM.JK
Date                   
2021-12-20  2185.601807
2021-12-21  2224.982178
2021-12-22  221

delete for mean variance

In [708]:
results_return5=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return5[results_return5.columns[i]]=np.array(result) 
results_return5.to_csv("BATTdeletemeanquarter.csv", index=True)

swap for mean variance

In [709]:
results_return6=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    # call the function and generate a table with stock adjusted close prices
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 
results_return6.to_csv("BATTswapmeanquarter.csv", index=True)

delete for risk parity

In [710]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
result

[0.259462378681077, 0.15361753974499653, 1.49502705916469]

In [711]:
results_return7=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]


    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 
results_return7.to_csv("BATTdeleteriskquarter.csv", index=True)

swap for risk parity

In [712]:
results_return8=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']


    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 
results_return8.to_csv("BATTswapriskquarter.csv", index=True)

yearly

In [713]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [714]:
# call the function and generate a table with stock adjusted close prices
stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']
portfolio=combine_stocks(stocks)

             300750.SZ
Date                  
2020-06-19  165.823349
2020-06-22  165.243637
2020-06-23  168.811966
2020-06-24  171.500732
2020-06-29  170.841034
              1211.HK
Date                 
2020-06-19  59.666733
2020-06-22  59.218117
2020-06-23  60.264896
2020-06-24  60.863060
2020-06-26  59.730934
                  BHP
Date                 
2020-06-19  41.948509
2020-06-22  42.954361
2020-06-23  43.575371
2020-06-24  42.342106
2020-06-25  43.216763
                  TSLA
Date                  
2020-06-19  200.179993
2020-06-22  198.863998
2020-06-23  200.356003
2020-06-24  192.169998
2020-06-25  197.195999
                GLEN.L
Date                  
2020-06-19  154.618149
2020-06-22  149.421661
2020-06-23  154.851913
2020-06-24  149.853180
2020-06-25  150.662354
                   LI
Date                 
2020-07-30  16.459999
2020-07-31  16.000000
2020-08-03  16.360001
2020-08-04  16.690001
2020-08-05  16.750000
             NIO
Date            
2020-06-19  7.34
2020

               FSF.DU
Date                 
2021-04-22  56.700001
2021-04-23  60.400002
2021-04-26  60.700001
2021-04-27  60.599998
2021-04-28  59.200001
              6121.TWO
Date                  
2020-06-19  267.995300
2020-06-22  264.727081
2020-06-23  265.544098
2020-06-24  265.952637
2020-06-29  261.458832
             LICY
Date             
2020-11-16  9.715
2020-11-17  9.720
2020-11-18  9.650
2020-11-19  9.650
2020-11-20  9.680
               020150.KS
Date                    
2020-06-19  47550.136719
2020-06-22  48891.671875
2020-06-23  48196.058594
2020-06-24  48345.121094
2020-06-25  52667.859375
              ZEV
Date             
2020-07-02   9.80
2020-07-06   9.80
2020-07-07   9.85
2020-07-08  10.00
2020-07-09   9.90
               ANTM.JK
Date                  
2020-06-19  593.458740
2020-06-22  591.328003
2020-06-23  581.554016
2020-06-24  591.328003
2020-06-25  586.440979
            FREY
Date            
2020-06-19  9.81
2020-06-22  9.88
2020-06-23  9.82
2020-06-24  

In [715]:
# call the function and generate a table with stock adjusted close prices
stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]
portfolio2=combine_stocks(stocks)

             300750.SZ
Date                  
2020-06-19  165.823349
2020-06-22  165.243637
2020-06-23  168.811966
2020-06-24  171.500732
2020-06-29  170.841034
              1211.HK
Date                 
2020-06-19  59.666737
2020-06-22  59.218113
2020-06-23  60.264900
2020-06-24  60.863060
2020-06-26  59.730934
                  BHP
Date                 
2020-06-19  41.948509
2020-06-22  42.954361
2020-06-23  43.575371
2020-06-24  42.342106
2020-06-25  43.216763
                  TSLA
Date                  
2020-06-19  200.179993
2020-06-22  198.863998
2020-06-23  200.356003
2020-06-24  192.169998
2020-06-25  197.195999
                GLEN.L
Date                  
2020-06-19  154.618164
2020-06-22  149.421661
2020-06-23  154.851898
2020-06-24  149.853195
2020-06-25  150.662354
                   LI
Date                 
2020-07-30  16.459999
2020-07-31  16.000000
2020-08-03  16.360001
2020-08-04  16.690001
2020-08-05  16.750000
             NIO
Date            
2020-06-19  7.34
2020

               FSF.DU
Date                 
2021-04-22  56.700001
2021-04-23  60.400002
2021-04-26  60.700001
2021-04-27  60.599998
2021-04-28  59.200001
              6121.TWO
Date                  
2020-06-19  267.995300
2020-06-22  264.727081
2020-06-23  265.544128
2020-06-24  265.952637
2020-06-29  261.458832
             LICY
Date             
2020-11-16  9.715
2020-11-17  9.720
2020-11-18  9.650
2020-11-19  9.650
2020-11-20  9.680
               020150.KS
Date                    
2020-06-19  47550.136719
2020-06-22  48891.671875
2020-06-23  48196.058594
2020-06-24  48345.121094
2020-06-25  52667.859375
              ZEV
Date             
2020-07-02   9.80
2020-07-06   9.80
2020-07-07   9.85
2020-07-08  10.00
2020-07-09   9.90
               ANTM.JK
Date                  
2020-06-19  593.458740
2020-06-22  591.328003
2020-06-23  581.554016
2020-06-24  591.328003
2020-06-25  586.440979
            FREY
Date            
2020-06-19  9.81
2020-06-22  9.88
2020-06-23  9.82
2020-06-24  

delete for mean variance

In [716]:
results_return9=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result) 
results_return9.to_csv("BATTdeletemeanyear.csv", index=True)

swap for mean variance

In [717]:
results_return10=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    # call the function and generate a table with stock adjusted close prices
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']

    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 
results_return10.to_csv("BATTswapmeanyear.csv", index=True)

In [718]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,253)
sd=portfolio_sd(df,cleaned_weights,253)
sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
result=[returns,sd,sharpe]
result

[5.081509067788713, 1.018317352648667, 4.960839619053039]

delete for risk pariity

In [719]:
results_return11=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY"]


    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result) 
results_return11.to_csv("BATTdeleteriskyear.csv", index=True)

swap for risk parity

In [721]:
results_return12=pd.DataFrame(columns=stocks)
for i in range(85):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["300750.SZ", "1211.HK", "BHP","TSLA","GLEN.L","LI","NIO","002812.SZ","300014.SZ","051910.KS","1772.HK","LCID","XPEV","603799.SS","006400.KS","002371.SZ","SQM","002709.SZ","300450.SZ","SEDG","ALB","603659.SS","6752.T","3993.HK","PLUG","S32.AX","6762.T","FM.TO","CHPT","6619.T","6701.T","5713.T","UMI.BR","MIN.AX","HYLN","361610.KS","VAR1.DE","IGO.AX","LAC","LUN.TO","FCEL","INCO.JK","LTHM","4004.T","BLNK","AKE.AX","EVGO","JMAT.L","NKLA","PLS.AX","PTRA","ENS","ERA.V","ARI.JO","2362.HK","FSF.DU","6121.TWO","LICY","020150.KS","ZEV","ANTM.JK","FREY","WBX","6674.T","LGO","3211.TWO","SLI","PLL","AMG.AS","NMG","1585.HK","LEV","VLTA","MVST","LTR.AX","TLOFF","VULNF","MNS.AX","QS","9R9.F","GSCCF","1958.HK","FSR","WOPEF","NILSY",'^TNX']


    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 
results_return12.to_csv("BATTswapriskyear.csv", index=True)

In [722]:
portfolio1=portfolio2
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio1)
returns=portfolio_returns(df,cleaned_weights,253)
sd=portfolio_sd(df,cleaned_weights,253)
sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
result=[returns,sd,sharpe]
result

[2.0122309713646516, 0.35184527560518164, 5.634382806348121]

# AIEQ

monthly

In [723]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [726]:
stocks = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP",'^TNX']
portfolio=combine_stocks(stocks)

                  AAPL
Date                  
2022-04-19  166.922546
2022-04-20  166.753021
2022-04-21  165.945343
2022-04-22  161.328537
2022-04-25  162.415436
                   BKNG
Date                   
2022-04-19  2239.379883
2022-04-20  2251.800049
2022-04-21  2239.320068
2022-04-22  2213.870117
2022-04-25  2246.040039
                    DG
Date                  
2022-04-19  254.787766
2022-04-20  258.459534
2022-04-21  255.256714
2022-04-22  248.252441
2022-04-25  247.933167
                  TMUS
Date                  
2022-04-19  132.839996
2022-04-20  132.699997
2022-04-21  132.100006
2022-04-22  128.630005
2022-04-25  128.809998
                   KO
Date                 
2022-04-19  64.603241
2022-04-20  65.486862
2022-04-21  65.735069
2022-04-22  64.781952
2022-04-25  65.467010
                   VLO
Date                  
2022-04-19  105.940285
2022-04-20  106.264778
2022-04-21  103.157562
2022-04-22  101.328636
2022-04-25   99.194878
                CMCSA
Date        

                   ZTS
Date                  
2022-04-19  185.274994
2022-04-20  187.436020
2022-04-21  184.381622
2022-04-22  181.616699
2022-04-25  179.969727
                 BLDR
Date                 
2022-04-19  63.630001
2022-04-20  64.989998
2022-04-21  62.389999
2022-04-22  61.410000
2022-04-25  65.849998
             BLI
Date            
2022-04-19  5.82
2022-04-20  5.68
2022-04-21  5.36
2022-04-22  5.29
2022-04-25  5.65
                   DOV
Date                  
2022-04-19  151.110016
2022-04-20  154.935608
2022-04-21  143.618286
2022-04-22  142.462646
2022-04-25  141.705505
                  PBF
Date                 
2022-04-19  29.490000
2022-04-20  28.990000
2022-04-21  27.410000
2022-04-22  26.639999
2022-04-25  25.770000
                  NOG
Date                 
2022-04-19  27.916067
2022-04-20  28.968754
2022-04-21  27.290413
2022-04-22  25.274420
2022-04-25  24.956625
                   LW
Date                 
2022-04-19  67.613655
2022-04-20  68.686417
2022-04-2

                    FN
Date                  
2022-04-19   99.730003
2022-04-20  102.760002
2022-04-21  100.870003
2022-04-22   97.660004
2022-04-25   99.550003
                 CTVA
Date                 
2022-04-19  60.828083
2022-04-20  61.007168
2022-04-21  59.902828
2022-04-22  57.385735
2022-04-25  57.007675
                 ONTO
Date                 
2022-04-19  71.949997
2022-04-20  74.769997
2022-04-21  73.500000
2022-04-22  71.510002
2022-04-25  73.010002
                 COOP
Date                 
2022-04-19  45.660000
2022-04-20  46.299999
2022-04-21  46.320000
2022-04-22  44.869999
2022-04-25  44.520000
                  SMP
Date                 
2022-04-19  42.520344
2022-04-20  43.052834
2022-04-21  42.934502
2022-04-22  42.106182
2022-04-25  42.037159
             ^TNX
Date             
2022-04-19  2.913
2022-04-20  2.840
2022-04-21  2.917
2022-04-22  2.906
2022-04-25  2.826


/Users/beckybian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


In [727]:
stocks = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP"]
portfolio2=combine_stocks(stocks)

                  AAPL
Date                  
2022-04-19  166.922546
2022-04-20  166.753021
2022-04-21  165.945343
2022-04-22  161.328537
2022-04-25  162.415436
                   BKNG
Date                   
2022-04-19  2239.379883
2022-04-20  2251.800049
2022-04-21  2239.320068
2022-04-22  2213.870117
2022-04-25  2246.040039
                    DG
Date                  
2022-04-19  254.787766
2022-04-20  258.459534
2022-04-21  255.256714
2022-04-22  248.252441
2022-04-25  247.933167
                  TMUS
Date                  
2022-04-19  132.839996
2022-04-20  132.699997
2022-04-21  132.100006
2022-04-22  128.630005
2022-04-25  128.809998
                   KO
Date                 
2022-04-19  64.603241
2022-04-20  65.486862
2022-04-21  65.735069
2022-04-22  64.781952
2022-04-25  65.467010
                   VLO
Date                  
2022-04-19  105.940285
2022-04-20  106.264778
2022-04-21  103.157562
2022-04-22  101.328636
2022-04-25   99.194878
                CMCSA
Date        

                   ZTS
Date                  
2022-04-19  185.940002
2022-04-20  187.779999
2022-04-21  184.720001
2022-04-22  181.949997
2022-04-25  180.300003
                 BLDR
Date                 
2022-04-19  63.630001
2022-04-20  64.989998
2022-04-21  62.389999
2022-04-22  61.410000
2022-04-25  65.849998
             BLI
Date            
2022-04-19  5.82
2022-04-20  5.68
2022-04-21  5.36
2022-04-22  5.29
2022-04-25  5.65
                   DOV
Date                  
2022-04-19  151.110016
2022-04-20  154.935608
2022-04-21  143.618286
2022-04-22  142.462646
2022-04-25  141.705505
                  PBF
Date                 
2022-04-19  29.490000
2022-04-20  28.990000
2022-04-21  27.410000
2022-04-22  26.639999
2022-04-25  25.770000
                  NOG
Date                 
2022-04-19  27.916067
2022-04-20  28.968754
2022-04-21  27.290413
2022-04-22  25.274420
2022-04-25  24.956625
                   LW
Date                 
2022-04-19  68.070000
2022-04-20  69.150002
2022-04-2

                    FN
Date                  
2022-04-19   99.730003
2022-04-20  102.760002
2022-04-21  100.870003
2022-04-22   97.660004
2022-04-25   99.550003
                 CTVA
Date                 
2022-04-19  61.139999
2022-04-20  61.320000
2022-04-21  60.209999
2022-04-22  57.680000
2022-04-25  57.299999
                 ONTO
Date                 
2022-04-19  71.949997
2022-04-20  74.769997
2022-04-21  73.500000
2022-04-22  71.510002
2022-04-25  73.010002
                 COOP
Date                 
2022-04-19  45.660000
2022-04-20  46.299999
2022-04-21  46.320000
2022-04-22  44.869999
2022-04-25  44.520000
                  SMP
Date                 
2022-04-19  43.119999
2022-04-20  43.660000
2022-04-21  43.540001
2022-04-22  42.700001
2022-04-25  42.630001


/Users/beckybian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


In [728]:
len(stocks)

114

In [729]:
results_return1=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return1[results_return1.columns[i]]=np.array(result) 
results_return1.to_csv("AIEQdeletemeanmonth.csv", index=True)

swap for mean variance

In [733]:
results_return2=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks1 = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP",'^TNX']

    if i!=0:
        stock=stocks1[0:i]+stocks1[i+1:]
    else:
        stock=stocks1[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return2[results_return2.columns[i]]=np.array(result)
results_return2.to_csv("AIEQswapmeanmonth.csv", index=True)
        

In [735]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
result

[0.1329421259241076, 0.13124141533663541, 0.7858961720242587]

delete for risk parity

In [736]:
results_return3=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return3[results_return3.columns[i]]=np.array(result) 
results_return3.to_csv("AIEQdeleteriskmonth.csv", index=True)

swap for risk parity

In [737]:
results_return4=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks1[0:i]+stocks1[i+1:]
    else:
        stock=stocks1[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,20)
    sd=portfolio_sd(df,cleaned_weights,20)
    sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
    result=[returns,sd,sharpe]
    results_return4[results_return4.columns[i]]=np.array(result) 
results_return4.to_csv("AIEQswapriskmonth.csv", index=True)

In [738]:
portfolio1=portfolio2
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio1)
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
result

[-0.0907253371050738, 0.09797176746171254, -1.2302047847832815]

quarterly

In [739]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [740]:
stocks1 = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP",'^TNX']
portfolio=combine_stocks(stocks1)

                  AAPL
Date                  
2021-12-20  169.050461
2021-12-21  172.277100
2021-12-22  174.916183
2021-12-23  175.553558
2021-12-27  179.586868
                   BKNG
Date                   
2021-12-20  2208.750000
2021-12-21  2365.750000
2021-12-22  2385.179932
2021-12-23  2402.250000
2021-12-27  2394.510010
                    DG
Date                  
2021-12-20  218.980957
2021-12-21  220.292465
2021-12-22  218.802124
2021-12-23  219.904984
2021-12-27  223.243347
                  TMUS
Date                  
2021-12-20  118.989998
2021-12-21  121.099998
2021-12-22  120.559998
2021-12-23  119.910004
2021-12-27  118.580002
                   KO
Date                 
2021-12-20  56.693283
2021-12-21  56.919899
2021-12-22  57.323864
2021-12-23  57.363274
2021-12-27  57.786949
                  VLO
Date                 
2021-12-20  66.777809
2021-12-21  69.334595
2021-12-22  69.315147
2021-12-23  69.733185
2021-12-27  71.094208
                CMCSA
Date               

                  NHI
Date                 
2021-12-20  52.829975
2021-12-21  54.024574
2021-12-22  53.584965
2021-12-23  53.518063
2021-12-27  54.425957
                   ZTS
Date                  
2021-12-20  233.047897
2021-12-21  236.410431
2021-12-22  240.320099
2021-12-23  241.255219
2021-12-27  245.234528
                 BLDR
Date                 
2021-12-20  75.129997
2021-12-21  78.690002
2021-12-22  81.059998
2021-12-23  82.150002
2021-12-27  84.309998
                  BLI
Date                 
2021-12-20  19.160000
2021-12-21  20.629999
2021-12-22  20.340000
2021-12-23  19.090000
2021-12-27  18.740000
                   DOV
Date                  
2021-12-20  167.266983
2021-12-21  170.136856
2021-12-22  171.288788
2021-12-23  172.530090
2021-12-27  177.386032
              PBF
Date             
2021-12-20  11.58
2021-12-21  12.51
2021-12-22  12.44
2021-12-23  12.33
2021-12-27  12.94
                  NOG
Date                 
2021-12-20  18.210125
2021-12-21  19.420853
20

                 SNEX
Date                 
2021-12-20  58.110001
2021-12-21  58.840000
2021-12-22  60.169998
2021-12-23  61.259998
2021-12-27  62.119999
                 PINS
Date                 
2021-12-20  35.759998
2021-12-21  36.740002
2021-12-22  37.259998
2021-12-23  37.419998
2021-12-27  36.930000
                    FN
Date                  
2021-12-20  113.769997
2021-12-21  116.260002
2021-12-22  117.459999
2021-12-23  118.019997
2021-12-27  122.029999
                 CTVA
Date                 
2021-12-20  44.411385
2021-12-21  45.463135
2021-12-22  45.572277
2021-12-23  46.078300
2021-12-27  46.643860
                  ONTO
Date                  
2021-12-20   90.529999
2021-12-21   93.989998
2021-12-22   96.709999
2021-12-23   99.110001
2021-12-27  102.269997
                 COOP
Date                 
2021-12-20  39.139999
2021-12-21  40.349998
2021-12-22  41.419998
2021-12-23  41.650002
2021-12-27  41.889999
                  SMP
Date                 
2021-12-20  49.042

/Users/beckybian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


In [741]:
stocks = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP"]
portfolio2=combine_stocks(stocks)

                  AAPL
Date                  
2021-12-20  169.050461
2021-12-21  172.277100
2021-12-22  174.916183
2021-12-23  175.553543
2021-12-27  179.586868
                   BKNG
Date                   
2021-12-20  2208.750000
2021-12-21  2365.750000
2021-12-22  2385.179932
2021-12-23  2402.250000
2021-12-27  2394.510010
                    DG
Date                  
2021-12-20  218.980957
2021-12-21  220.292465
2021-12-22  218.802139
2021-12-23  219.904984
2021-12-27  223.243347
                  TMUS
Date                  
2021-12-20  118.989998
2021-12-21  121.099998
2021-12-22  120.559998
2021-12-23  119.910004
2021-12-27  118.580002
                   KO
Date                 
2021-12-20  56.693283
2021-12-21  56.919899
2021-12-22  57.323864
2021-12-23  57.363274
2021-12-27  57.786949
                  VLO
Date                 
2021-12-20  66.777809
2021-12-21  69.334595
2021-12-22  69.315155
2021-12-23  69.733192
2021-12-27  71.094208
                CMCSA
Date               

                  NHI
Date                 
2021-12-20  52.829975
2021-12-21  54.024574
2021-12-22  53.584965
2021-12-23  53.518063
2021-12-27  54.425961
                   ZTS
Date                  
2021-12-20  233.884384
2021-12-21  237.258972
2021-12-22  241.182678
2021-12-23  242.121170
2021-12-27  246.114746
                 BLDR
Date                 
2021-12-20  75.129997
2021-12-21  78.690002
2021-12-22  81.059998
2021-12-23  82.150002
2021-12-27  84.309998
                  BLI
Date                 
2021-12-20  19.160000
2021-12-21  20.629999
2021-12-22  20.340000
2021-12-23  19.090000
2021-12-27  18.740000
                   DOV
Date                  
2021-12-20  167.897903
2021-12-21  170.778595
2021-12-22  171.934875
2021-12-23  173.180847
2021-12-27  178.055115
              PBF
Date             
2021-12-20  11.58
2021-12-21  12.51
2021-12-22  12.44
2021-12-23  12.33
2021-12-27  12.94
                  NOG
Date                 
2021-12-20  18.375208
2021-12-21  19.596910
20

                 SNEX
Date                 
2021-12-20  58.110001
2021-12-21  58.840000
2021-12-22  60.169998
2021-12-23  61.259998
2021-12-27  62.119999
                 PINS
Date                 
2021-12-20  35.759998
2021-12-21  36.740002
2021-12-22  37.259998
2021-12-23  37.419998
2021-12-27  36.930000
                    FN
Date                  
2021-12-20  113.769997
2021-12-21  116.260002
2021-12-22  117.459999
2021-12-23  118.019997
2021-12-27  122.029999
                 CTVA
Date                 
2021-12-20  44.411385
2021-12-21  45.463135
2021-12-22  45.572277
2021-12-23  46.078300
2021-12-27  46.643860
                  ONTO
Date                  
2021-12-20   90.529999
2021-12-21   93.989998
2021-12-22   96.709999
2021-12-23   99.110001
2021-12-27  102.269997
                 COOP
Date                 
2021-12-20  39.139999
2021-12-21  40.349998
2021-12-22  41.419998
2021-12-23  41.650002
2021-12-27  41.889999
                  SMP
Date                 
2021-12-20  49.042

/Users/beckybian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


delete for mean variance

In [742]:
results_return5=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return5[results_return5.columns[i]]=np.array(result) 
results_return5.to_csv("AIEQdeletemeanquarter.csv", index=True)

swap for mean variance

In [743]:
results_return6=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks1 = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP",'^TNX']

    if i!=0:
        stock=stocks1[0:i]+stocks1[i+1:]
    else:
        stock=stocks1[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result)
results_return6.to_csv("AIEQswapmeanquarter.csv", index=True)

In [745]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
result

[0.483347465473717, 0.15738549908079338, 2.8817614591093283]

delete for risk parity

In [746]:
results_return7=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 
results_return7.to_csv("AIEQdeleteriskquarter.csv", index=True)

swap for risk parity

In [747]:
results_return8=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks1[0:i]+stocks1[i+1:]
    else:
        stock=stocks1[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 
results_return8.to_csv("AIEQswapriskquarter.csv", index=True)

In [748]:
portfolio1=portfolio2
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio1)
returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
result

[0.0452645910607865, 0.11022146790208708, 0.14030470973698286]

yearly

In [749]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [750]:
stocks1 = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP",'^TNX']
portfolio=combine_stocks(stocks1)

                 AAPL
Date                 
2020-06-19  86.231712
2020-06-22  88.487862
2020-06-23  90.376610
2020-06-24  88.781281
2020-06-25  89.959892
                   BKNG
Date                   
2020-06-19  1626.910034
2020-06-22  1633.520020
2020-06-23  1677.099976
2020-06-24  1610.329956
2020-06-25  1615.390015
                    DG
Date                  
2020-06-19  185.450439
2020-06-22  188.751190
2020-06-23  186.619431
2020-06-24  188.466263
2020-06-25  187.307083
                  TMUS
Date                  
2020-06-19  106.900002
2020-06-22  106.599998
2020-06-23  107.160004
2020-06-24  108.430000
2020-06-25  110.190002
                   KO
Date                 
2020-06-19  43.264805
2020-06-22  43.001572
2020-06-23  42.907555
2020-06-24  41.976826
2020-06-25  42.240063
                  VLO
Date                 
2020-06-19  56.108540
2020-06-22  55.664368
2020-06-23  55.655483
2020-06-24  51.542400
2020-06-25  50.885014
                CMCSA
Date                 
2020

                   ZTS
Date                  
2020-06-19  135.580078
2020-06-22  135.126190
2020-06-23  136.280685
2020-06-24  132.521133
2020-06-25  133.517776
                 BLDR
Date                 
2020-06-19  21.580000
2020-06-22  21.700001
2020-06-23  21.700001
2020-06-24  20.660000
2020-06-25  20.309999
                  BLI
Date                 
2020-07-17  65.449997
2020-07-20  62.099998
2020-07-21  58.500000
2020-07-22  56.000000
2020-07-23  58.099998
                  DOV
Date                 
2020-06-19  92.232872
2020-06-22  92.524437
2020-06-23  92.135674
2020-06-24  89.307457
2020-06-25  89.696220
              PBF
Date             
2020-06-19  12.88
2020-06-22  12.67
2020-06-23  12.06
2020-06-24  10.69
2020-06-25  10.55
                 NOG
Date                
2020-06-19  9.806633
2020-06-22  9.512435
2020-06-23  9.316302
2020-06-24  8.433704
2020-06-25  8.531771
                   LW
Date                 
2020-06-19  64.159676
2020-06-22  63.820465
2020-06-23  64.4

                   FN
Date                 
2020-06-19  60.189999
2020-06-22  60.410000
2020-06-23  60.380001
2020-06-24  58.560001
2020-06-25  59.200001
                 CTVA
Date                 
2020-06-19  26.379171
2020-06-22  26.816719
2020-06-23  26.661148
2020-06-24  25.348509
2020-06-25  25.474915
                 ONTO
Date                 
2020-06-19  33.950001
2020-06-22  33.830002
2020-06-23  33.580002
2020-06-24  33.279999
2020-06-25  34.450001
             COOP
Date             
2020-06-19  13.08
2020-06-22  13.21
2020-06-23  13.14
2020-06-24  12.18
2020-06-25  12.75
                  SMP
Date                 
2020-06-19  38.083328
2020-06-22  38.675098
2020-06-23  38.675098
2020-06-24  37.491558
2020-06-25  37.825619
             ^TNX
Date             
2020-06-19  0.697
2020-06-22  0.704
2020-06-23  0.709
2020-06-24  0.684
2020-06-25  0.674


/Users/beckybian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


In [751]:
stocks = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP"]
portfolio2=combine_stocks(stocks)

                 AAPL
Date                 
2020-06-19  86.231705
2020-06-22  88.487854
2020-06-23  90.376595
2020-06-24  88.781281
2020-06-25  89.959900
                   BKNG
Date                   
2020-06-19  1626.910034
2020-06-22  1633.520020
2020-06-23  1677.099976
2020-06-24  1610.329956
2020-06-25  1615.390015
                    DG
Date                  
2020-06-19  185.450439
2020-06-22  188.751190
2020-06-23  186.619476
2020-06-24  188.466293
2020-06-25  187.307098
                  TMUS
Date                  
2020-06-19  106.900002
2020-06-22  106.599998
2020-06-23  107.160004
2020-06-24  108.430000
2020-06-25  110.190002
                   KO
Date                 
2020-06-19  43.264805
2020-06-22  43.001568
2020-06-23  42.907555
2020-06-24  41.976830
2020-06-25  42.240063
                  VLO
Date                 
2020-06-19  56.108543
2020-06-22  55.664364
2020-06-23  55.655479
2020-06-24  51.542397
2020-06-25  50.885014
                CMCSA
Date                 
2020

                   ZTS
Date                  
2020-06-19  136.066742
2020-06-22  135.611206
2020-06-23  136.769836
2020-06-24  132.996826
2020-06-25  133.997025
                 BLDR
Date                 
2020-06-19  21.580000
2020-06-22  21.700001
2020-06-23  21.700001
2020-06-24  20.660000
2020-06-25  20.309999
                  BLI
Date                 
2020-07-17  65.449997
2020-07-20  62.099998
2020-07-21  58.500000
2020-07-22  56.000000
2020-07-23  58.099998
                  DOV
Date                 
2020-06-19  92.580757
2020-06-22  92.873421
2020-06-23  92.483200
2020-06-24  89.644325
2020-06-25  90.034554
              PBF
Date             
2020-06-19  12.88
2020-06-22  12.67
2020-06-23  12.06
2020-06-24  10.69
2020-06-25  10.55
                 NOG
Date                
2020-06-19  9.895535
2020-06-22  9.598667
2020-06-23  9.400759
2020-06-24  8.510160
2020-06-25  8.609115
                   LW
Date                 
2020-06-19  64.592705
2020-06-22  64.251213
2020-06-23  64.9

                   FN
Date                 
2020-06-19  60.189999
2020-06-22  60.410000
2020-06-23  60.380001
2020-06-24  58.560001
2020-06-25  59.200001
                 CTVA
Date                 
2020-06-19  26.379169
2020-06-22  26.816719
2020-06-23  26.661148
2020-06-24  25.348509
2020-06-25  25.474913
                 ONTO
Date                 
2020-06-19  33.950001
2020-06-22  33.830002
2020-06-23  33.580002
2020-06-24  33.279999
2020-06-25  34.450001
             COOP
Date             
2020-06-19  13.08
2020-06-22  13.21
2020-06-23  13.14
2020-06-24  12.18
2020-06-25  12.75
                  SMP
Date                 
2020-06-19  38.620403
2020-06-22  39.220528
2020-06-23  39.220528
2020-06-24  38.020287
2020-06-25  38.359070


/Users/beckybian/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


delete for mean variance

In [752]:
results_return9=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result) 
results_return9.to_csv("AIEQdeletemeanyear.csv", index=True)

swap for mean variance

In [757]:
results_return10=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks1 = ["AAPL", "BKNG", "DG","TMUS","KO","VLO","CMCSA","VEEV","FTNT","AMD","PATH","PSX","FDX","RBLX","ZM","MPC","VZ","EXAS","DHR","BEAM","UBER","NTLA","PD","PACB","DNA","XPEV","BEKE","ROKU","TWOU","AZO","NVR","CHWY","CERS","MAR","U","CDEV","PSTG","F","MRTX","EPRT","HASI","CAT","IAA","RAMP","TWST","VCYT","ARCH","ITT","VICI","RETA","OXY","DDS","UAA","NHI","ZTS","BLDR","BLI","DOV","PBF","NOG","LW","GH","BKR","GOOGL","GSAT","SGFY","FATE","FND","CE","NFG","APD","OSK","ACI","PYPL","EL","DCPH","CRM","OKE","HCC","TWTR","SWTX","TEX","TXG","WING","GKOS","COUP","HCSG","ATGE","FLT","NRIX","OI","ARR","RGNX","MHR.F","TENB","UHS","SGMO","EXTR","NVDA","CHS","TSLA","EVRI","IART","KN","WWD","GME","AEP","SNEX","PINS","FN","CTVA","ONTO","COOP","SMP",'^TNX']

    if i!=0:
        stock=stocks1[0:i]+stocks1[i+1:]
    else:
        stock=stocks1[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result)
results_return10.to_csv("AIEQswapmeanyear.csv", index=True)

In [754]:
portfolio1=portfolio2
# calculate the expected return using historical data
mu = mean_historical_return(portfolio1, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio1).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
cleaned_weights=list(cleaned_weights.values())
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()

returns=portfolio_returns(df,cleaned_weights,253)
sd=portfolio_sd(df,cleaned_weights,253)
sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
result=[returns,sd,sharpe]
result

[0.147237206376736, 0.12340828500897234, 0.951615253126626]

delete for risk parity

In [756]:
results_return11=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result) 
results_return11.to_csv("AIEQdeleteriskyear.csv", index=True)

swap for risk parity

In [758]:
results_return12=pd.DataFrame(columns=stocks)
for i in range(114):
    #results_volatility=pd.DataFrame(columns=stocks)
    if i!=0:
        stock=stocks1[0:i]+stocks1[i+1:]
    else:
        stock=stocks1[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 
results_return12.to_csv("AIEQswapriskyear.csv", index=True)

In [759]:
portfolio1=portfolio2
df=pd.DataFrame(columns=portfolio1.columns)
for j in df:
    df[j]=portfolio1[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio1)
returns=portfolio_returns(df,cleaned_weights,253)
sd=portfolio_sd(df,cleaned_weights,253)
sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
result=[returns,sd,sharpe]
result

[0.14634247691925023, 0.13784768530231575, 0.8454438437878681]